In [ ]:
A.<x,z>=AffineSpace(QQ,2)
t=3
#(x^2- t*z^2)^2,4*x*z*(x-z)*(x-t*z)
f=DynamicalSystem_affine([(x^2- t*z^2)^2/(4*x*z*(x-z)*(x-t*z)), (x^2+z^2)^2/(4*x*z*(x-z)*(x-t*z))])
F=f.homogenize(1)
F.is_morphism()

In [3]:
def sigma(self,n, chow=False):
    #print(self, n)
    d = self.degree()
    N = self.domain().dimension_relative()
    F2 = self.nth_iterate_map(n)
    P2 = F2.domain()
    T = 1
    f = F2
    for j in range(N,-1,-1):
        #print("j:",j)
        X = f.periodic_points(1,return_scheme=True, minimal=False)
        Xa = X.affine_patch(j)
        fa = f.dehomogenize(j)
        Pa = fa.domain()
        #print('Pa coordinate ring:', Pa.coordinate_ring())
        if chow:
            R = PolynomialRing(self.base_ring(),'v', N+N+3, order='lex')
            #print('R:', R)
            im = [R.gen(i) for i in range(j)] + (N-j)*[0]
            R_zero = {R.gen(N):1}
            for j in range(N+1,N+N+1):
                R_zero.update({R.gen(j):0})
            #print('R_zero:', R_zero)
            vars = list(R.gens())
            t = vars.pop()
            w = vars.pop()
            vars = vars[:N]
        else:
            R = PolynomialRing(self.base_ring(),'v', N+2, order='lex')
            im = list(R.gens())[:j] + (N-j)*[0]
            vars = list(R.gens())
            t = vars.pop()
            w = vars.pop()
        #print("im:",im)
        #print t,w

        phi = Pa.coordinate_ring().hom(im,R)
        #print('phi:', phi)
        MS = MatrixSpace(R,N,N)
        M = t*MS.one()
        #print(list(fa))
        try:
            g = (M-jacobian([phi(F.numerator())/phi(F.denominator()) for F in fa], vars)).det()
            #print "g:",g
            if chow:
                I = R.ideal([phi(h) for h in Xa.defining_polynomials()] + [w*g.denominator()-R.gen(N)*g.numerator() + sum(R.gen(j-1)*R.gen(N+j)*g.denominator() for j in range(1,N+1))])
            else:
                I = R.ideal([phi(h) for h in Xa.defining_polynomials()] + [w*g.denominator()-g.numerator()])
            #print I
            G = I.groebner_basis()
            #print "G:",G[-1]
            T = T*G[-1]
            #print "T:",T
            #print T.coefficient(R.gen(2*N+1)**8)
            #print T.specialization(R_zero)
        except ZeroDivisionError:
            #what to do about multiplicities!?
            print('Zero Division Error')
            print(j, j==0)
            infin = self.domain()([1] + N*[0])
            if self(infin) == infin:
                T *= w-self.multiplier(infin,1).characteristic_polynomial(t)

        #print('T:', T)
        if chow:
            T2 = T.specialization(R_zero)
            newR = PolynomialRing(self.base_ring(), 'w,t',2, order='lex')
            psi = T2.parent().hom(N*[0]+list(newR.gens()), newR)
            #print('newR gens:', newR.gens())
            #print(psi(T2).degrees())
            #print('T2:', T2)
            #print('psi(T2):',psi(T2))
            #print('psi:', psi)
        else:
            T2 = T
            newR = PolynomialRing(self.base_ring(), 'w,t',2, order='lex')
            psi = T2.parent().hom(N*[0]+list(newR.gens()), newR)
            print(psi(T2).degrees())
            #print psi(T2)
    if chow:
        #print('R_zero:', R_zero)
        T = T.specialization(R_zero)
    return psi(T)

def num_sigma(self, n):
    fc = self.change_ring(CC)
    per= self.periodic_points(n,minimal=False,return_scheme=True).rational_points(F=CC)
    mul = []
    t=1
    for Q in per:
        t=t*fc.multiplier(Q,n,check=False).characteristic_polynomial()
        c=fc.multiplier(Q,n, check=False).characteristic_polynomial().coefficients(sparse=False)
        c.pop(-1) #always monic
        mul.append(c)
    sig=[]
    e = SymmetricFunctions(CC).e()

    N = len(mul)
    R = CC
    for i in range(0,N):
        T=[]
        for k in range(len(mul[0])):
            T.append((R(e([i+1]).expand(N)([m[k] for m in mul]))))
        sig.append(T)
    return sig,t

In [17]:
P.<x,y,z>=ProjectiveSpace(QQ,2)
f=DynamicalSystem([x^2,z^2,y^2])
#f=DynamicalSystem([x^4-2*x^3*y-x^2*y^2+3*x*y^3,y^4])
#for Q in f.periodic_points(1):
#    print Q,f.multiplier(Q,1).charpoly().factor()
sigma_polynomial = f.sigma_invariants(1, return_polynomial=True)
sigma_polynomial

ZeroDivisionError: 

In [9]:
P.<x,y,z>=ProjectiveSpace(QQ,2)
f=DynamicalSystem([x^2-2*x*y,y^2,z^2])
#f=DynamicalSystem([x^4-2*x^3*y-x^2*y^2+3*x*y^3,y^4])
#for Q in f.periodic_points(1):
#    print Q,f.multiplier(Q,1).charpoly().factor()
sigma_polynomial = (f.sigma_invariants(1, return_polynomial=True))
sigmas = []
sigma_dictionary = dict([list(reversed(i)) for i in list(sigma_polynomial)])
degree = sigma_polynomial.degree()
print(sigma_polynomial)
for i in range(degree, -1, -1):
    for j in range(degree-i, -1, -1):
        print(i, j)
        sigmas.append(sigma_dictionary.pop(w**i*t**j, 0))
sigmas

w^7 - 7*w^6*t^2 + 10*w^6*t - 4*w^6 + 21*w^5*t^4 - 60*w^5*t^3 + 44*w^5*t^2 + 8*w^5*t - 32*w^5 - 35*w^4*t^6 + 150*w^4*t^5 - 160*w^4*t^4 - 80*w^4*t^3 + 272*w^4*t^2 - 128*w^4*t + 35*w^3*t^8 - 200*w^3*t^7 + 280*w^3*t^6 + 240*w^3*t^5 - 864*w^3*t^4 + 480*w^3*t^3 + 128*w^3*t^2 - 21*w^2*t^10 + 150*w^2*t^9 - 260*w^2*t^8 - 320*w^2*t^7 + 1280*w^2*t^6 - 672*w^2*t^5 - 768*w^2*t^4 + 512*w^2*t^3 + 7*w*t^12 - 60*w*t^11 + 124*w*t^10 + 200*w*t^9 - 896*w*t^8 + 416*w*t^7 + 1152*w*t^6 - 1024*w*t^5 - t^14 + 10*t^13 - 24*t^12 - 48*t^11 + 240*t^10 - 96*t^9 - 512*t^8 + 512*t^7
14 0
13 1
13 0
12 2
12 1
12 0
11 3
11 2
11 1
11 0
10 4
10 3
10 2
10 1
10 0
9 5
9 4
9 3
9 2
9 1
9 0
8 6
8 5
8 4
8 3
8 2
8 1
8 0
7 7
7 6
7 5
7 4
7 3
7 2
7 1
7 0
6 8
6 7
6 6
6 5
6 4
6 3
6 2
6 1
6 0
5 9
5 8
5 7
5 6
5 5
5 4
5 3
5 2
5 1
5 0
4 10
4 9
4 8
4 7
4 6
4 5
4 4
4 3
4 2
4 1
4 0
3 11
3 10
3 9
3 8
3 7
3 6
3 5
3 4
3 3
3 2
3 1
3 0
2 12
2 11
2 10
2 9
2 8
2 7
2 6
2 5
2 4
2 3
2 2
2 1
2 0
1 13
1 12
1 11
1 10
1 9
1 8
1 7
1 6
1 5
1 4
1 3
1 2
1 1
1

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 -7,
 10,
 -4,
 0,
 0,
 0,
 0,
 0,
 21,
 -60,
 44,
 8,
 -32,
 0,
 0,
 0,
 0,
 -35,
 150,
 -160,
 -80,
 272,
 -128,
 0,
 0,
 0,
 0,
 35,
 -200,
 280,
 240,
 -864,
 480,
 128,
 0,
 0,
 0,
 0,
 -21,
 150,
 -260,
 -320,
 1280,
 -672,
 -768,
 512,
 0,
 0,
 0,
 0,
 7,
 -60,
 124,
 200,
 -896,
 416,
 1152,
 -1024,
 0,
 0,
 0,
 0,
 0,
 -1,
 10,
 -24,
 -48,
 240,
 -96,
 -512,
 512,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [1]:
P.<w,t> = ProjectiveSpace(QQ,1)
f=DynamicalSystem([w^2-2*w*t,t^2])
sigmas = f.sigma_invariants(1, return_polynomial=True)
#sigma(f,1,chow=True)
sigmas

w^3 - 3*w^2*t + 2*w^2 + 3*w*t^2 - 4*w*t - 8*w - t^3 + 2*t^2 + 8*t

In [11]:
R.<w,t>=QQ[]

((w-(t-1))*(w-(t-2))*(w-(t-3))).specialization({w:0})

-t^3 + 6*t^2 - 11*t + 6

In [4]:
f.sigma_invariants(1)

[2, -8, 0]

In [3]:
sigma(f,1)

Dynamical System of Projective Space of dimension 1 over Rational Field
  Defn: Defined on coordinates by sending (x : y) to
        (x^2 - 2*x*y : y^2) 1
j: 1
im: [v0]
[x^2 - 2*x]
(2, 2)
j: 0
im: [0]
[y^2/(-2*y + 1)]
(3, 3)


w^3 - 3*w^2*t + 2*w^2 + 3*w*t^2 - 4*w*t - 8*w - t^3 + 2*t^2 + 8*t

In [11]:
f.sigma_invariants?

In [34]:
P.<x,y>=ProjectiveSpace(QQ,1)
f=DynamicalSystem([x^3,y^3])
for Q in f.periodic_points(1):
    print Q,f.multiplier(Q,1).charpoly().factor()

(-1 : 1) x - 3
(0 : 1) x
(1 : 0) x
(1 : 1) x - 3


In [14]:
(x-1)*x*(x+1)*(x-2) + x

x^4 - 2*x^3 - x^2 + 3*x

In [38]:
P.<x,y,z,w>=ProjectiveSpace(QQ,3)
f=DynamicalSystem([x^3-3*x*y^2,y^3,z^3,w^3])
for Q in f.periodic_points(1):
    print Q,f.multiplier(Q,1).charpoly().factor()

(-2 : -1 : -1 : 1) (x - 9) * (x - 3)^2
(-2 : -1 : 0 : 1) (x - 9) * (x - 3) * x
(-2 : -1 : 1 : 0) (x - 9) * (x - 3) * x
(-2 : -1 : 1 : 1) (x - 9) * (x - 3)^2
(-2 : 1 : -1 : 1) (x - 9) * (x - 3)^2
(-2 : 1 : 0 : 0) (x - 9) * x^2
(-2 : 1 : 0 : 1) (x - 9) * (x - 3) * x
(-2 : 1 : 1 : 0) (x - 9) * (x - 3) * x
(-2 : 1 : 1 : 1) (x - 9) * (x - 3)^2
(-1 : 0 : -1 : 1) x * (x - 3)^2
(-1 : 0 : 0 : 1) (x - 3) * x^2
(-1 : 0 : 1 : 0) (x - 3) * x^2
(-1 : 0 : 1 : 1) x * (x - 3)^2
(0 : -1 : -1 : 1) (x + 3) * (x - 3)^2
(0 : -1 : 0 : 1) (x - 3) * x * (x + 3)
(0 : -1 : 1 : 0) (x - 3) * x * (x + 3)
(0 : -1 : 1 : 1) (x + 3) * (x - 3)^2
(0 : 0 : -1 : 1) (x - 3) * x^2
(0 : 0 : 0 : 1) x^3
(0 : 0 : 1 : 0) x^3
(0 : 0 : 1 : 1) (x - 3) * x^2
(0 : 1 : -1 : 1) (x + 3) * (x - 3)^2
(0 : 1 : 0 : 0) (x + 3) * x^2
(0 : 1 : 0 : 1) (x - 3) * x * (x + 3)
(0 : 1 : 1 : 0) (x - 3) * x * (x + 3)
(0 : 1 : 1 : 1) (x + 3) * (x - 3)^2
(1 : 0 : -1 : 1) x * (x - 3)^2
(1 : 0 : 0 : 0) x^3
(1 : 0 : 0 : 1) (x - 3) * x^2
(1 : 0 : 1 : 0) (x -

In [10]:
P.<x,y,z,w>=ProjectiveSpace(QQ,3)
f=DynamicalSystem([x^3-3*x*y^2,y^3,z^3,w^3])
f.periodic_points(1)

[(-2 : -1 : -1 : 1),
 (-2 : -1 : 0 : 1),
 (-2 : -1 : 1 : 0),
 (-2 : -1 : 1 : 1),
 (-2 : 1 : -1 : 1),
 (-2 : 1 : 0 : 0),
 (-2 : 1 : 0 : 1),
 (-2 : 1 : 1 : 0),
 (-2 : 1 : 1 : 1),
 (-1 : 0 : -1 : 1),
 (-1 : 0 : 0 : 1),
 (-1 : 0 : 1 : 0),
 (-1 : 0 : 1 : 1),
 (0 : -1 : -1 : 1),
 (0 : -1 : 0 : 1),
 (0 : -1 : 1 : 0),
 (0 : -1 : 1 : 1),
 (0 : 0 : -1 : 1),
 (0 : 0 : 0 : 1),
 (0 : 0 : 1 : 0),
 (0 : 0 : 1 : 1),
 (0 : 1 : -1 : 1),
 (0 : 1 : 0 : 0),
 (0 : 1 : 0 : 1),
 (0 : 1 : 1 : 0),
 (0 : 1 : 1 : 1),
 (1 : 0 : -1 : 1),
 (1 : 0 : 0 : 0),
 (1 : 0 : 0 : 1),
 (1 : 0 : 1 : 0),
 (1 : 0 : 1 : 1),
 (2 : -1 : -1 : 1),
 (2 : -1 : 0 : 1),
 (2 : -1 : 1 : 0),
 (2 : -1 : 1 : 1),
 (2 : 1 : -1 : 1),
 (2 : 1 : 0 : 0),
 (2 : 1 : 0 : 1),
 (2 : 1 : 1 : 0),
 (2 : 1 : 1 : 1)]

In [9]:
3^3+3^2+3+1

40

In [20]:
S.<a,b>=QQ[]
P1.<u,v>=ProjectiveSpace(S,1)
F = P1.Lattes_map(EllipticCurve(S,[0,a]),2)
G = P1.Lattes_map(EllipticCurve(S,[b,1]),2)
F,G
P3.<x,y,z,w>=ProjectiveSpace(S,3)
phi=P1.coordinate_ring().hom([z,w],P3.coordinate_ring())
phi(G[0])

z^4 + (-2*b)*z^2*w^2 + (-8)*z*w^3 + b^2*w^4

In [68]:
S1.<a,b>=QQ[]
S=FractionField(S1)
P1.<x,y>=ProjectiveSpace(S,1)
F = P1.Lattes_map(EllipticCurve(S,[0,a]),2)
G = P1.Lattes_map(EllipticCurve(S,[b,1]),2)
F,G
P3.<x,y,z,w>=ProjectiveSpace(S,3)
phi=P1.coordinate_ring().hom([z,w],P3.coordinate_ring())
R=P3.coordinate_ring()
f=DynamicalSystem([R(F[0]),R(F[1]),phi(G[0]),phi(G[1])])
print f.is_morphism()
(P1*P1).segre_embedding()

R.<u0,u1,u2,u3,x0,x1,x2,x3>=PolynomialRing(S,order='lex')
x=x0*x2
y=x0*x3
z=x1*x2
w=x1*x3
I=R.ideal([u0-x0*x2, u1-x0*x3, u2-x1*x2,u3-x1*x3, 1/4*x^4 + (-2*a)*x*y^3])#, x^3*y + y^4, 1/4*z^4 - 2*z*w^3, z^3*w + w^4])
print I.groebner_basis()
I=R.ideal([u0-x0*x2, u1-x0*x3, u2-x1*x2,u3-x1*x3, x^3*y + a*y^4])
print I.groebner_basis()
I=R.ideal([u0-x0*x2, u1-x0*x3, u2-x1*x2,u3-x1*x3, 1/4*z^4 + (-1/2*b)*z^2*w^2 + (-2)*z*w^3 + 1/4*b^2*w^4])
print I.groebner_basis()
I=R.ideal([u0-x0*x2, u1-x0*x3, u2-x1*x2,u3-x1*x3, z^3*w + b*z*w^3 + w^4])
I.groebner_basis()

True
[u0 - x0*x2, u1 - x0*x3, u2 - x1*x2, u3 - x1*x3, x0^4*x2^4 + (-8*a)*x0^4*x2*x3^3]
[u0 - x0*x2, u1 - x0*x3, u2 - x1*x2, u3 - x1*x3, x0^4*x2^3*x3 + a*x0^4*x3^4]
[u0 - x0*x2, u1 - x0*x3, u2 - x1*x2, u3 - x1*x3, x1^4*x2^4 + (-2*b)*x1^4*x2^2*x3^2 + (-8)*x1^4*x2*x3^3 + b^2*x1^4*x3^4]


[u0 - x0*x2, u1 - x0*x3, u2 - x1*x2, u3 - x1*x3, x1^4*x2^3*x3 + b*x1^4*x2*x3^3 + x1^4*x3^4]

In [ ]:
R.<a,b>=GF(7)[]
P.<x,y,z,w>=ProjectiveSpace(FractionField(R),3)
F=DynamicalSystem([x^4 + (-8*a)*x*y^3, x^3*y + a*y^4, z^4+ (-2*b)*z^2*w^2 - 8*z*w^3 + b^2*w^4, z^3*w + b*z*w^3 + w^4])
s=sigma(F,1)

Dynamical System of Projective Space of dimension 3 over Fraction Field of Multivariate Polynomial Ring in a, b over Finite Field of size 7
  Defn: Defined on coordinates by sending (x : y : z : w) to
        (x^4 + (-a)*x*y^3 : x^3*y + a*y^4 : z^4 + (-2*b)*z^2*w^2 - z*w^3 + b^2*w^4 : z^3*w + b*z*w^3 + w^4) 1
j: 3
im: [v0, v1, v2]
[(x0^4 + (-a)*x0*x1^3)/(x2^3 + b*x2 + 1), (x0^3*x1 + a*x1^4)/(x2^3 + b*x2 + 1), (x2^4 + (-2*b)*x2^2 - x2 + b^2)/(x2^3 + b*x2 + 1)]


In [74]:
s

w^4 - w^3*t^3 - w^3*t - w^2*t^4 - w^2*t^3 - w*t^9 - w*t^7 - w*t^6 + t^12 - t^9

In [33]:
_.groebner_basis()

[x0*x2 - u0, x0*x3 - u1, x0*u2 - x1*u0, x0*u3 - x1*u1, x1*x2 - u2, x1*x3 - u3, x2*u1 - x3*u0, x2*u3 - x3*u2, u0^4 + (-8*a)*u0*u1^3, u0^3*u1*u2 + (-8*a)*u1^4*u2, u0^2*u1^2*u2^2 + (-8*a)*u1^4*u2*u3, u0*u1^3*u2^3 + (-8*a)*u1^4*u2*u3^2, u0*u3 - u1*u2, u1^4*u2^4 + (-8*a)*u1^4*u2*u3^3]

In [64]:
P1.<a,b>=ProjectiveSpace(QQ,1)
F = P1.Lattes_map(EllipticCurve(QQ,[0,1]),2)
G = P1.Lattes_map(EllipticCurve(QQ,[0,2]),2)
P3.<x,y,z,w>=ProjectiveSpace(QQ,3)
phi=P1.coordinate_ring().hom([z,w],P3.coordinate_ring())
R=P3.coordinate_ring()
f=DynamicalSystem([R(F[0]),R(F[1]),phi(G[0]),phi(G[1])])
f.is_morphism()
psi=(P1*P1).segre_embedding()

R.<u0,u1,u2,u3,x0,x1,x2,x3>=PolynomialRing(QQ, order='lex')
x=x0*x2
y=x0*x3
z=x1*x2
w=x1*x3
I=R.ideal([u0-x0*x2, u1-x0*x3, u2-x1*x2,u3-x1*x3, 1/4*x^4 - 2*x*y^3])#, x^3*y + y^4, 1/4*z^4 - 2*z*w^3, z^3*w + w^4])
I.groebner_basis()

[u0 - x0*x2, u1 - x0*x3, u2 - x1*x2, u3 - x1*x3, x0^4*x2^4 - 8*x0^4*x2*x3^3]

In [62]:
I.elimination_ideal([x0,x1,x2,x3])

Ideal (u1*u2 - u0*u3, u0^4 - 8*u0*u1^3) of Multivariate Polynomial Ring in u0, u1, u2, u3, x0, x1, x2, x3 over Rational Field

In [ ]:
u0^4 - 8*u0*u1^3

In [4]:
R.<a>=QQ[]
P.<u,v,z,w>=ProjectiveSpace(FractionField(R),3)
f=DynamicalSystem([(u^2 - a*v^2)^2 , 4*u*v*(u-v)*(u - a*v), z^4,w^4])
f

Dynamical System of Projective Space of dimension 3 over Fraction Field of Univariate Polynomial Ring in a over Rational Field
  Defn: Defined on coordinates by sending (u : v : z : w) to
        (u^4 + (-2*a)*u^2*v^2 + a^2*v^4 : 4*u^3*v + (-4*a - 4)*u^2*v^2 + 4*a*u*v^3 : z^4 : w^4)

In [5]:
sigma(f,1)

Dynamical System of Projective Space of dimension 3 over Fraction Field of Univariate Polynomial Ring in a over Rational Field
  Defn: Defined on coordinates by sending (u : v : z : w) to
        (u^4 + (-2*a)*u^2*v^2 + a^2*v^4 : 4*u^3*v + (-4*a - 4)*u^2*v^2 + 4*a*u*v^3 : z^4 : w^4) 1
j: 3
im: [v0, v1, v2]
[x0^4 + (-2*a)*x0^2*x1^2 + a^2*x1^4, 4*x0^3*x1 + (-4*a - 4)*x0^2*x1^2 + 4*a*x0*x1^3, x2^4]
(6, 18)
j: 2
im: [v0, v1, 0]
[x0^4 + (-2*a)*x0^2*x1^2 + a^2*x1^4, 4*x0^3*x1 + (-4*a - 4)*x0^2*x1^2 + 4*a*x0*x1^3, x2^4]
(9, 27)
j: 1
im: [v0, 0, 0]
[(x0^4 + (-2*a)*x0^2 + a^2)/(4*x0^3 + (-4*a - 4)*x0^2 + 4*a*x0), x1^4/(4*x0^3 + (-4*a - 4)*x0^2 + 4*a*x0), x2^4/(4*x0^3 + (-4*a - 4)*x0^2 + 4*a*x0)]
(10, 30)
j: 0
im: [0, 0, 0]
[(4*a*x0^3 + (-4*a - 4)*x0^2 + 4*x0)/(a^2*x0^4 + (-2*a)*x0^2 + 1), x1^4/(a^2*x0^4 + (-2*a)*x0^2 + 1), x2^4/(a^2*x0^4 + (-2*a)*x0^2 + 1)]
(11, 33)


w^11 + (-11)*w^10*t^3 + 40*w^10*t^2 + (-64)*w^10*t + 32*w^10 + 55*w^9*t^6 + (-400)*w^9*t^5 + 1272*w^9*t^4 + (-2080)*w^9*t^3 + 1280*w^9*t^2 + 1024*w^9*t + (-2048)*w^9 + (-165)*w^8*t^9 + 1800*w^8*t^8 + (-8568)*w^8*t^7 + 22240*w^8*t^6 + (-28864)*w^8*t^5 + (-128)*w^8*t^4 + 59392*w^8*t^3 + (-75776)*w^8*t^2 + 32768*w^8*t + 330*w^7*t^12 + (-4800)*w^7*t^11 + 30432*w^7*t^10 + (-106880)*w^7*t^9 + 204112*w^7*t^8 + (-104320)*w^7*t^7 + (-450048)*w^7*t^6 + 1122304*w^7*t^5 + (-987136)*w^7*t^4 + 32768*w^7*t^3 + 393216*w^7*t^2 + (-462)*w^6*t^15 + 8400*w^6*t^14 + (-66528)*w^6*t^13 + 293440*w^6*t^12 + (-728112)*w^6*t^11 + 673408*w^6*t^10 + 1613056*w^6*t^9 + (-6323200)*w^6*t^8 + 8130560*w^6*t^7 + (-917504)*w^6*t^6 + (-9633792)*w^6*t^5 + 10747904*w^6*t^4 + (-4194304)*w^6*t^3 + 462*w^5*t^18 + (-10080)*w^5*t^17 + 95760*w^5*t^16 + (-507584)*w^5*t^15 + 1537424*w^5*t^14 + (-1971584)*w^5*t^13 + (-3345664)*w^5*t^12 + 18916352*w^5*t^11 + (-30838784)*w^5*t^10 + 5046272*w^5*t^9 + 58785792*w^5*t^8 + (-88866816)*w^5*t

In [21]:
P.<x,y,z>=ProjectiveSpace(QQ,2)
f=DynamicalSystem([x^2+y^2,y^2+z^2,z^2])
s=sigma(f,1)
for p,e in s.factor():
    for q, r in p.subs({s.parent().gen(0):0}).univariate_polynomial().factor():
        print q.roots(ring=CC, multiplicities=False)

Dynamical System of Projective Space of dimension 2 over Rational Field
  Defn: Defined on coordinates by sending (x : y : z) to
        (x^2 + y^2 : y^2 + z^2 : z^2) 1
j: 2
im: [v0, v1]
[x0^2 + x1^2, x1^2 + 1]
(4, 8)
j: 1
im: [v0, 0]
[(x0^2 + 1)/(x1^2 + 1), x1^2/(x1^2 + 1)]
(6, 12)
j: 0
im: [0, 0]
[(x0^2 + x1^2)/(x0^2 + 1), x1^2/(x0^2 + 1)]
(7, 14)
[0.000000000000000]
[0.000000000000000]
[1.00000000000000 - 1.73205080756888*I, 1.00000000000000 + 1.73205080756888*I]
[1.00000000000000 - 1.73205080756888*I, 1.00000000000000 + 1.73205080756888*I]
[-0.947122966707013 - 0.889543617524132*I, -0.947122966707013 + 0.889543617524132*I, 2.94712296670701 - 0.889543617524132*I, 2.94712296670701 + 0.889543617524132*I]


In [42]:
P.<x,y,z>=ProjectiveSpace(QQ,2)
f=DynamicalSystem([x^2-x*z+2*z^2,x^2+x*y+y^2,z^2])
s=sigma(f,1)
for p,e in s.factor():
    print p
    for q, r in p.subs({s.parent().gen(0):0}).univariate_polynomial().factor():
        print q.roots(ring=CC)

Dynamical System of Projective Space of dimension 2 over Rational Field
  Defn: Defined on coordinates by sending (x : y : z) to
        (x^2 - x*z + 2*z^2 : x^2 + x*y + y^2 : z^2) 1
j: 2
im: [v0, v1]
[x0^2 - x0 + 2, x0^2 + x0*x1 + x1^2]
(4, 8)
j: 1
im: [v0, 0]
[(x0^2 - x0*x1 + 2*x1^2)/(x0^2 + x0 + 1), x1^2/(x0^2 + x0 + 1)]
(7, 14)
j: 0
im: [0, 0]
[(x0^2 + x0 + 1)/(2*x1^2 - x1 + 1), x1^2/(2*x1^2 - x1 + 1)]
(7, 14)
-w + t^2
[(0.000000000000000, 1)]
w^2 - 2*w*t^2 + 2*w*t + t^4 - 2*t^3 + 5*t^2
[(0.000000000000000, 1)]
[(1.00000000000000 - 2.00000000000000*I, 1), (1.00000000000000 + 2.00000000000000*I, 1)]
w^4 - 4*w^3*t^2 + 8*w^3*t - 4*w^3 + 6*w^2*t^4 - 24*w^2*t^3 + 46*w^2*t^2 + 20*w^2*t - 56*w^2 - 4*w*t^6 + 24*w*t^5 - 80*w*t^4 + 96*w*t^3 - 4*w*t^2 - 312*w*t + 280*w + t^8 - 8*t^7 + 38*t^6 - 116*t^5 + 317*t^4 - 644*t^3 + 1312*t^2 - 1560*t + 1700
[(1.00000000000000 - 2.00000000000000*I, 1), (1.00000000000000 + 2.00000000000000*I, 1)]
[(-0.879129818333282 - 2.12864484453120*I, 1), (-0.8791298

In [59]:
set_verbose(None)
P.<x,y,z>=ProjectiveSpace(QQ,2)
f1=DynamicalSystem([x^2-y*z+2*y^2,x^2+z*y+y^2+x*z,z^2])
f1bar=f1.change_ring(QQbar)
for Q in f1bar.periodic_points(1):
    print f1bar.multiplier(Q,1).eigenvalues()

[1.570759536769330? + 1.427686444349533?*I, -4.772793948534994? - 7.888238201338854?*I]
[1.570759536769330? - 1.427686444349533?*I, -4.772793948534994? + 7.888238201338854?*I]
[0, -6.831177207208337? + 0.8496965970360521?*I]
[0, -6.831177207208337? - 0.8496965970360521?*I]
[1.713414949255797?, -0.3093461257244682?]
[0.50000000000000000? + 0.866025403784439?*I, 0.50000000000000000? - 0.866025403784439?*I]
[0, -0.3376455855833262?]


In [67]:
set_verbose(None)
P.<x,y,z>=ProjectiveSpace(QQ,2)
f2=DynamicalSystem([x^2-2*x*z+z^2,x^2+x*y+y^2+x*z,z^2])
f2=DynamicalSystem([x*(z-(x-y)),y*(z+(x-y)),z^2])
f2bar=f2.change_ring(QQbar)
for Q in f2bar.periodic_points(1):
    #print Q,
    print f2bar.multiplier(Q,1).eigenvalues()

TypeError: use return_scheme=True

In [92]:
r.<a1,a2,a3,b1,b2,b3,c1,c2,c3>=QQ[]
m=matrix(r,3,3,list(r.gens()))
P.<x,y,z>=ProjectiveSpace(FractionField(r),2)
f2=DynamicalSystem([x*(z-(x-y)),y*(z+(x-y)),z^2])
g=f2.conjugate(m)
g.scale_by(-a3*b2*c1 + a2*b3*c1 + a3*b1*c2 - a1*b3*c2 - a2*b1*c3 + a1*b2*c3)
g[2]

(a1*a2*b1*c1 - a2*b1^2*c1 + a1^2*b2*c1 - a1*b1*b2*c1 - 2*a1^2*b1*c2 + 2*a1*b1^2*c2)*x^2 + (a2^2*b1*c1 + 3*a1*a2*b2*c1 - 3*a2*b1*b2*c1 - a1*b2^2*c1 - 3*a1*a2*b1*c2 + a2*b1^2*c2 - a1^2*b2*c2 + 3*a1*b1*b2*c2)*x*y + (2*a2^2*b2*c1 - 2*a2*b2^2*c1 - a2^2*b1*c2 - a1*a2*b2*c2 + a2*b1*b2*c2 + a1*b2^2*c2)*y^2 + (a2*a3*b1*c1 + 2*a1*a3*b2*c1 - a3*b1*b2*c1 + a1*a2*b3*c1 - 2*a2*b1*b3*c1 - a1*b2*b3*c1 - a3*b2*c1^2 + a2*b3*c1^2 - 3*a1*a3*b1*c2 + a3*b1^2*c2 - a1^2*b3*c2 + 3*a1*b1*b3*c2 + a3*b1*c1*c2 - a1*b3*c1*c2 - a2*b1*c1*c3 + a1*b2*c1*c3)*x*z + (3*a2*a3*b2*c1 - a3*b2^2*c1 + a2^2*b3*c1 - 3*a2*b2*b3*c1 - 2*a2*a3*b1*c2 - a1*a3*b2*c2 + a3*b1*b2*c2 - a1*a2*b3*c2 + a2*b1*b3*c2 + 2*a1*b2*b3*c2 - a3*b2*c1*c2 + a2*b3*c1*c2 + a3*b1*c2^2 - a1*b3*c2^2 - a2*b1*c2*c3 + a1*b2*c2*c3)*y*z + (a3^2*b2*c1 + a2*a3*b3*c1 - a3*b2*b3*c1 - a2*b3^2*c1 - a3^2*b1*c2 - a1*a3*b3*c2 + a3*b1*b3*c2 + a1*b3^2*c2 - a3*b2*c1*c3 + a2*b3*c1*c3 + a3*b1*c2*c3 - a1*b3*c2*c3 - a2*b1*c3^2 + a1*b2*c3^2)*z^2

In [77]:
g[0].monomials()

[x^2, x*y, y^2, x*z, y*z, z^2]

In [88]:
([g[0].coefficient(x*y), (g[0].coefficient(y*z)), (g[0].coefficient(y^2))])

[a2*a3*b1*c2 + a1*a3*b2*c2 - 2*a3*b1*b2*c2 + 2*a1*a2*b3*c2 - a2*b1*b3*c2 - a1*b2*b3*c2 - a3*b2*c1*c2 + a2*b3*c1*c2 + a3*b1*c2^2 - a1*b3*c2^2 - a2^2*b1*c3 - 3*a1*a2*b2*c3 + 3*a2*b1*b2*c3 + a1*b2^2*c3 - a2*b1*c2*c3 + a1*b2*c2*c3,
 a3^2*b2*c2 + 3*a2*a3*b3*c2 - 3*a3*b2*b3*c2 - a2*b3^2*c2 - 3*a2*a3*b2*c3 + a3*b2^2*c3 - a2^2*b3*c3 + 3*a2*b2*b3*c3,
 a2*a3*b2*c2 - a3*b2^2*c2 + a2^2*b3*c2 - a2*b2*b3*c2 - 2*a2^2*b2*c3 + 2*a2*b2^2*c3]

In [93]:
g[2].coefficients()

[a1*a2*b1*c1 - a2*b1^2*c1 + a1^2*b2*c1 - a1*b1*b2*c1 - 2*a1^2*b1*c2 + 2*a1*b1^2*c2,
 a2^2*b1*c1 + 3*a1*a2*b2*c1 - 3*a2*b1*b2*c1 - a1*b2^2*c1 - 3*a1*a2*b1*c2 + a2*b1^2*c2 - a1^2*b2*c2 + 3*a1*b1*b2*c2,
 2*a2^2*b2*c1 - 2*a2*b2^2*c1 - a2^2*b1*c2 - a1*a2*b2*c2 + a2*b1*b2*c2 + a1*b2^2*c2,
 a2*a3*b1*c1 + 2*a1*a3*b2*c1 - a3*b1*b2*c1 + a1*a2*b3*c1 - 2*a2*b1*b3*c1 - a1*b2*b3*c1 - a3*b2*c1^2 + a2*b3*c1^2 - 3*a1*a3*b1*c2 + a3*b1^2*c2 - a1^2*b3*c2 + 3*a1*b1*b3*c2 + a3*b1*c1*c2 - a1*b3*c1*c2 - a2*b1*c1*c3 + a1*b2*c1*c3,
 3*a2*a3*b2*c1 - a3*b2^2*c1 + a2^2*b3*c1 - 3*a2*b2*b3*c1 - 2*a2*a3*b1*c2 - a1*a3*b2*c2 + a3*b1*b2*c2 - a1*a2*b3*c2 + a2*b1*b3*c2 + 2*a1*b2*b3*c2 - a3*b2*c1*c2 + a2*b3*c1*c2 + a3*b1*c2^2 - a1*b3*c2^2 - a2*b1*c2*c3 + a1*b2*c2*c3,
 a3^2*b2*c1 + a2*a3*b3*c1 - a3*b2*b3*c1 - a2*b3^2*c1 - a3^2*b1*c2 - a1*a3*b3*c2 + a3*b1*b3*c2 + a1*b3^2*c2 - a3*b2*c1*c3 + a2*b3*c1*c3 + a3*b1*c2*c3 - a1*b3*c2*c3 - a2*b1*c3^2 + a1*b2*c3^2]

In [98]:
r.<a1,a2,a3,b1,b2,b3,c1,c2,c3>=QQ[]
I=r.ideal([a2*a3*b1*c2 + a1*a3*b2*c2 - 2*a3*b1*b2*c2 + 2*a1*a2*b3*c2 - a2*b1*b3*c2 - a1*b2*b3*c2 - a3*b2*c1*c2 + a2*b3*c1*c2 + a3*b1*c2^2 - a1*b3*c2^2 - a2^2*b1*c3 - 3*a1*a2*b2*c3 + 3*a2*b1*b2*c3 + a1*b2^2*c3 - a2*b1*c2*c3 + a1*b2*c2*c3, a3^2*b2*c2 + 3*a2*a3*b3*c2 - 3*a3*b2*b3*c2 - a2*b3^2*c2 - 3*a2*a3*b2*c3 + a3*b2^2*c3 - a2^2*b3*c3 + 3*a2*b2*b3*c3,  a2*a3*b2*c2 - a3*b2^2*c2 + a2^2*b3*c2 - a2*b2*b3*c2 - 2*a2^2*b2*c3 + 2*a2*b2^2*c3, a1*a2*b1*c1 - a2*b1^2*c1 + a1^2*b2*c1 - a1*b1*b2*c1 - 2*a1^2*b1*c2 + 2*a1*b1^2*c2,  a2^2*b1*c1 + 3*a1*a2*b2*c1 - 3*a2*b1*b2*c1 - a1*b2^2*c1 - 3*a1*a2*b1*c2 + a2*b1^2*c2 - a1^2*b2*c2 + 3*a1*b1*b2*c2,2*a2^2*b2*c1 - 2*a2*b2^2*c1 - a2^2*b1*c2 - a1*a2*b2*c2 + a2*b1*b2*c2 + a1*b2^2*c2,  a2*a3*b1*c1 + 2*a1*a3*b2*c1 - a3*b1*b2*c1 + a1*a2*b3*c1 - 2*a2*b1*b3*c1 - a1*b2*b3*c1 - a3*b2*c1^2 + a2*b3*c1^2 - 3*a1*a3*b1*c2 + a3*b1^2*c2 - a1^2*b3*c2 + 3*a1*b1*b3*c2 + a3*b1*c1*c2 - a1*b3*c1*c2 - a2*b1*c1*c3 + a1*b2*c1*c3,  3*a2*a3*b2*c1 - a3*b2^2*c1 + a2^2*b3*c1 - 3*a2*b2*b3*c1 - 2*a2*a3*b1*c2 - a1*a3*b2*c2 + a3*b1*b2*c2 - a1*a2*b3*c2 + a2*b1*b3*c2 + 2*a1*b2*b3*c2 - a3*b2*c1*c2 + a2*b3*c1*c2 + a3*b1*c2^2 - a1*b3*c2^2 - a2*b1*c2*c3 + a1*b2*c2*c3,  a3^2*b2*c1 + a2*a3*b3*c1 - a3*b2*b3*c1 - a2*b3^2*c1 - a3^2*b1*c2 - a1*a3*b3*c2 + a3*b1*b3*c2 + a1*b3^2*c2 - a3*b2*c1*c3 + a2*b3*c1*c3 + a3*b1*c2*c3 - a1*b3*c2*c3 - a2*b1*c3^2 + a1*b2*c3^2-1])

In [99]:
I.dimension()

-1

In [34]:
P1.<z,w>=ProjectiveSpace(QQ,1)
F = P1.Lattes_map(EllipticCurve(QQ,[0,2]),2)
P2.<x,y,z>=ProjectiveSpace(QQ,2)
R=P2.coordinate_ring()
f=DynamicalSystem([R(F[0]),x^4+y^4,R(F[1])])
f.is_morphism()

False

In [37]:
P1.<z,w>=ProjectiveSpace(QQ,1)
F = P1.Lattes_map(EllipticCurve(QQ,[0,1]),2)
G = P1.Lattes_map(EllipticCurve(QQ,[0,2]),2)
P2.<x,y,z>=ProjectiveSpace(QQ,2)
R=P2.coordinate_ring()
f=DynamicalSystem([R(F[0])*R(G[0]),R(F[0])*R(G[1]) + R(F[1])*R(G[0]),R(F[1])*R(G[1])])
f.is_morphism()

False

In [18]:
P1.<a,b>=ProjectiveSpace(QQ,1)
F = P1.Lattes_map(EllipticCurve(QQ,[0,1]),2)
G = P1.Lattes_map(EllipticCurve(QQ,[0,2]),2)
P3.<x,y,z,w>=ProjectiveSpace(QQ,3)
phi=P1.coordinate_ring().hom([z,w],P3.coordinate_ring())
R=P3.coordinate_ring()
f=DynamicalSystem([R(F[0]),R(F[1]),phi(G[0]),phi(G[1])])
f.is_morphism()

True

In [26]:
P1.<a,b>=ProjectiveSpace(QQ,1)
F = P1.Lattes_map(EllipticCurve(QQ,[0,1]),2)
P3.<x,y,z,w>=ProjectiveSpace(QQ,3)
phi=P1.coordinate_ring().hom([z,w],P3.coordinate_ring())
R=P3.coordinate_ring()
f=DynamicalSystem([R(F[0]),R(F[1]),z^4,w^4])
f.is_morphism()
s1=sigma(f,1)

P1.<a,b>=ProjectiveSpace(QQ,1)
F = P1.Lattes_map(EllipticCurve(QQ,[0,2]),2)
P3.<x,y,z,w>=ProjectiveSpace(QQ,3)
phi=P1.coordinate_ring().hom([z,w],P3.coordinate_ring())
R=P3.coordinate_ring()
g=DynamicalSystem([R(F[0]),R(F[1]),z^4,w^4])
g.is_morphism()
s2=sigma(g,1)

s1==s2

Dynamical System of Projective Space of dimension 3 over Rational Field
  Defn: Defined on coordinates by sending (x : y : z : w) to
        (1/4*x^4 - 2*x*y^3 : x^3*y + y^4 : z^4 : w^4) 1
j: 3
im: [v0, v1, v2]
[1/4*x0^4 - 2*x0*x1^3, x0^3*x1 + x1^4, x2^4]
(6, 18)
j: 2
im: [v0, v1, 0]
[1/4*x0^4 - 2*x0*x1^3, x0^3*x1 + x1^4, x2^4]
(9, 27)
j: 1
im: [v0, 0, 0]
[(1/4*x0^4 - 2*x0)/(x0^3 + 1), x1^4/(x0^3 + 1), x2^4/(x0^3 + 1)]
(10, 30)
j: 0
im: [0, 0, 0]
[(x0^4 + x0)/(-2*x0^3 + 1/4), x1^4/(-2*x0^3 + 1/4), x2^4/(-2*x0^3 + 1/4)]
(11, 33)
Dynamical System of Projective Space of dimension 3 over Rational Field
  Defn: Defined on coordinates by sending (x : y : z : w) to
        (1/4*x^4 - 4*x*y^3 : x^3*y + 2*y^4 : z^4 : w^4) 1
j: 3
im: [v0, v1, v2]
[1/4*x0^4 - 4*x0*x1^3, x0^3*x1 + 2*x1^4, x2^4]
(6, 18)
j: 2
im: [v0, v1, 0]
[1/4*x0^4 - 4*x0*x1^3, x0^3*x1 + 2*x1^4, x2^4]
(9, 27)
j: 1
im: [v0, 0, 0]
[(1/4*x0^4 - 4*x0)/(x0^3 + 2), x1^4/(x0^3 + 2), x2^4/(x0^3 + 2)]
(10, 30)
j: 0
im: [0, 0, 0]
[(2*x0^4 

True

In [25]:
P1.<a,b>=ProjectiveSpace(QQ,1)
F = P1.Lattes_map(EllipticCurve(QQ,[0,1]),2)
G = P1.Lattes_map(EllipticCurve(QQ,[0,2]),2)
P3.<x,y,z,w>=ProjectiveSpace(QQ,3)
phi=P1.coordinate_ring().hom([z,w],P3.coordinate_ring())
R=P3.coordinate_ring()
f=DynamicalSystem([R(F[0]),R(F[1]),phi(G[0]),phi(G[1])])
f.is_morphism()
s1=sigma(f,1)

P1.<a,b>=ProjectiveSpace(QQ,1)
F = P1.Lattes_map(EllipticCurve(QQ,[0,2]),2)
G = P1.Lattes_map(EllipticCurve(QQ,[0,3]),2)
P3.<x,y,z,w>=ProjectiveSpace(QQ,3)
phi=P1.coordinate_ring().hom([z,w],P3.coordinate_ring())
R=P3.coordinate_ring()
g=DynamicalSystem([R(F[0]),R(F[1]),phi(G[0]),phi(G[1])])
g.is_morphism()
s2=sigma(g,1)

s1==s2

Dynamical System of Projective Space of dimension 3 over Rational Field
  Defn: Defined on coordinates by sending (x : y : z : w) to
        (1/4*x^4 - 2*x*y^3 : x^3*y + y^4 : 1/4*z^4 - 4*z*w^3 : z^3*w + 2*w^4) 1
j: 3
im: [v0, v1, v2]
[(1/4*x0^4 - 2*x0*x1^3)/(x2^3 + 2), (x0^3*x1 + x1^4)/(x2^3 + 2), (1/4*x2^4 - 4*x2)/(x2^3 + 2)]
(3, 9)
j: 2
im: [v0, v1, 0]
[(1/4*x0^4 - 2*x0*x1^3)/(-4*x2^3 + 1/4), (x0^3*x1 + x1^4)/(-4*x2^3 + 1/4), (2*x2^4 + x2)/(-4*x2^3 + 1/4)]
(6, 18)
j: 1
im: [v0, 0, 0]
[(1/4*x0^4 - 2*x0)/(x0^3 + 1), (1/4*x1^4 - 4*x1*x2^3)/(x0^3 + 1), (x1^3*x2 + 2*x2^4)/(x0^3 + 1)]
(7, 21)
j: 0
im: [0, 0, 0]
[(x0^4 + x0)/(-2*x0^3 + 1/4), (1/4*x1^4 - 4*x1*x2^3)/(-2*x0^3 + 1/4), (x1^3*x2 + 2*x2^4)/(-2*x0^3 + 1/4)]
(8, 24)
Dynamical System of Projective Space of dimension 3 over Rational Field
  Defn: Defined on coordinates by sending (x : y : z : w) to
        (1/4*x^4 - 4*x*y^3 : x^3*y + 2*y^4 : 1/4*z^4 - 6*z*w^3 : z^3*w + 3*w^4) 1
j: 3
im: [v0, v1, v2]
[(1/4*x0^4 - 4*x0*x1^3)/(x2^3 + 3

True

In [19]:
f

Dynamical System of Projective Space of dimension 3 over Rational Field
  Defn: Defined on coordinates by sending (x : y : z : w) to
        (1/4*x^4 - 2*x*y^3 : x^3*y + y^4 : 1/4*z^4 - 4*z*w^3 : z^3*w + 2*w^4)

In [89]:
(P1*P1).segre_embedding()

Scheme morphism:
  From: Product of projective spaces P^1 x P^1 over Rational Field
  To:   Closed subscheme of Projective Space of dimension 3 over Rational Field defined by:
  -u1*u2 + u0*u3
  Defn: Defined by sending (x0 : x1 , x2 : x3) to 
        (x0*x2 : x0*x3 : x1*x2 : x1*x3).

In [20]:
R.<x0,x1,x2,x3,u0,u1,u2,u3>=QQ[]
x=x0*x2
y=x0*x3
z=x1*x2
w=x1*x3
I=R.ideal([u0-x0*x2, u1-x0*x3, u2-x1*x2,u3-x1*x3, 1/4*x^4 - 2*x*y^3])#, x^3*y + y^4, 1/4*z^4 - 2*z*w^3, z^3*w + w^4])
print I.elimination_ideal([x0,x1,x2,x3]).gen(1)
I=R.ideal([u0-x0*x2, u1-x0*x3, u2-x1*x2,u3-x1*x3, x^3*y + y^4])
print I.elimination_ideal([x0,x1,x2,x3]).gen(1)
I=R.ideal([u0-x0*x2, u1-x0*x3, u2-x1*x2,u3-x1*x3, 1/4*z^4 - 4*z*w^3])
print I.elimination_ideal([x0,x1,x2,x3]).gen(1)
I=R.ideal([u0-x0*x2, u1-x0*x3, u2-x1*x2,u3-x1*x3, z^3*w + 2*w^4])
I.elimination_ideal([x0,x1,x2,x3]).gen(1)

u0^4 - 8*u0*u1^3
u0^3*u1 + u1^4
u2^4 - 16*u2*u3^3


u2^3*u3 + 2*u3^4

In [21]:
P.<u0,u1,u2,u3>=ProjectiveSpace(QQ,3)
f=DynamicalSystem([u0^4 - 8*u0*u1^3,u0^3*u1 + u1^4,u2^4 - 8*u2*u3^3,u2^3*u3 + u3^4])
f.is_morphism()

True

In [14]:
s2=sigma(f,1)

Dynamical System of Projective Space of dimension 3 over Rational Field
  Defn: Defined on coordinates by sending (u0 : u1 : u2 : u3) to
        (u0^4 - 8*u0*u1^3 : u0^3*u1 + u1^4 : u2^4 - 8*u2*u3^3 : u2^3*u3 + u3^4) 1
j: 3
im: [v0, v1, v2]
[(x0^4 - 8*x0*x1^3)/(x2^3 + 1), (x0^3*x1 + x1^4)/(x2^3 + 1), (x2^4 - 8*x2)/(x2^3 + 1)]
(4, 12)
j: 2
im: [v0, v1, 0]
[(x0^4 - 8*x0*x1^3)/(-8*x2^3 + 1), (x0^3*x1 + x1^4)/(-8*x2^3 + 1), (x2^4 + x2)/(-8*x2^3 + 1)]
(8, 24)
j: 1
im: [v0, 0, 0]
[(x0^4 - 8*x0)/(x0^3 + 1), (x1^4 - 8*x1*x2^3)/(x0^3 + 1), (x1^3*x2 + x2^4)/(x0^3 + 1)]
(9, 27)
j: 0
im: [0, 0, 0]
[(x0^4 + x0)/(-8*x0^3 + 1), (x1^4 - 8*x1*x2^3)/(-8*x0^3 + 1), (x1^3*x2 + x2^4)/(-8*x0^3 + 1)]
(10, 30)


In [22]:
%%time
s1=sigma(f,1)

Dynamical System of Projective Space of dimension 3 over Rational Field
  Defn: Defined on coordinates by sending (u0 : u1 : u2 : u3) to
        (u0^4 - 8*u0*u1^3 : u0^3*u1 + u1^4 : u2^4 - 16*u2*u3^3 : u2^3*u3 + 2*u3^4) 1
j: 3
im: [v0, v1, v2]
[(x0^4 - 8*x0*x1^3)/(x2^3 + 2), (x0^3*x1 + x1^4)/(x2^3 + 2), (x2^4 - 16*x2)/(x2^3 + 2)]
(4, 12)
j: 2
im: [v0, v1, 0]
[(x0^4 - 8*x0*x1^3)/(-16*x2^3 + 1), (x0^3*x1 + x1^4)/(-16*x2^3 + 1), (2*x2^4 + x2)/(-16*x2^3 + 1)]
(8, 24)
j: 1
im: [v0, 0, 0]
[(x0^4 - 8*x0)/(x0^3 + 1), (x1^4 - 16*x1*x2^3)/(x0^3 + 1), (x1^3*x2 + 2*x2^4)/(x0^3 + 1)]
(9, 27)
j: 0
im: [0, 0, 0]
[(x0^4 + x0)/(-8*x0^3 + 1), (x1^4 - 16*x1*x2^3)/(-8*x0^3 + 1), (x1^3*x2 + 2*x2^4)/(-8*x0^3 + 1)]
(10, 30)
CPU times: user 64 ms, sys: 0 ns, total: 64 ms
Wall time: 80 ms


In [23]:
s1==s2

True

In [24]:
P.<u0,u1,u2,u3>=ProjectiveSpace(QQ,3)
f=DynamicalSystem([u0^4 - 8*u0*u1^3,u0^3*u1 + u1^4,u2^4 - 8*u2*u3^3,u2^3*u3 + u3^4])
f.is_morphism()

P.<u0,u1,u2,u3>=ProjectiveSpace(QQ,3)
g=DynamicalSystem([u0^4 - 8*u0*u1^3,u0^3*u1 + u1^4,u2^4 - 16*u2*u3^3,u2^3*u3 + 2*u3^4])
g.is_morphism()

s1=sigma(f,1)
s2=sigma(g,1)

s1==s2

Dynamical System of Projective Space of dimension 3 over Rational Field
  Defn: Defined on coordinates by sending (u0 : u1 : u2 : u3) to
        (u0^4 - 8*u0*u1^3 : u0^3*u1 + u1^4 : u2^4 - 8*u2*u3^3 : u2^3*u3 + u3^4) 1
j: 3
im: [v0, v1, v2]
[(x0^4 - 8*x0*x1^3)/(x2^3 + 1), (x0^3*x1 + x1^4)/(x2^3 + 1), (x2^4 - 8*x2)/(x2^3 + 1)]
(4, 12)
j: 2
im: [v0, v1, 0]
[(x0^4 - 8*x0*x1^3)/(-8*x2^3 + 1), (x0^3*x1 + x1^4)/(-8*x2^3 + 1), (x2^4 + x2)/(-8*x2^3 + 1)]
(8, 24)
j: 1
im: [v0, 0, 0]
[(x0^4 - 8*x0)/(x0^3 + 1), (x1^4 - 8*x1*x2^3)/(x0^3 + 1), (x1^3*x2 + x2^4)/(x0^3 + 1)]
(9, 27)
j: 0
im: [0, 0, 0]
[(x0^4 + x0)/(-8*x0^3 + 1), (x1^4 - 8*x1*x2^3)/(-8*x0^3 + 1), (x1^3*x2 + x2^4)/(-8*x0^3 + 1)]
(10, 30)
Dynamical System of Projective Space of dimension 3 over Rational Field
  Defn: Defined on coordinates by sending (u0 : u1 : u2 : u3) to
        (u0^4 - 8*u0*u1^3 : u0^3*u1 + u1^4 : u2^4 - 16*u2*u3^3 : u2^3*u3 + 2*u3^4) 1
j: 3
im: [v0, v1, v2]
[(x0^4 - 8*x0*x1^3)/(x2^3 + 2), (x0^3*x1 + x1^4)/(x2^3 + 2),

True

In [20]:
s1

w^12 - 12*w^11*t^2 - 92/3*w^11*t - 16*w^11 + 66*w^10*t^4 + 1012/3*w^10*t^3 + 592*w^10*t^2 + 4064/9*w^10*t + 128*w^10 - 220*w^9*t^6 - 5060/3*w^9*t^5 - 5040*w^9*t^4 - 210368/27*w^9*t^3 - 183680/27*w^9*t^2 - 83968/27*w^9*t - 2048/9*w^9 + 495*w^8*t^8 + 5060*w^8*t^7 + 21360*w^8*t^6 + 149408/3*w^8*t^5 + 5829376/81*w^8*t^4 + 5353472/81*w^8*t^3 + 2704384/81*w^8*t^2 + 237568/81*w^8*t - 1064960/729*w^8 - 792*w^7*t^10 - 10120*w^7*t^9 - 55200*w^7*t^8 - 516352/3*w^7*t^7 - 28041728/81*w^7*t^6 - 4247488/9*w^7*t^5 - 34071296/81*w^7*t^4 - 15681536/81*w^7*t^3 - 3350528/729*w^7*t^2 + 2375680/729*w^7*t - 14876672/729*w^7 + 924*w^6*t^12 + 14168*w^6*t^11 + 94752*w^6*t^10 + 3329984/9*w^6*t^9 + 77179648/81*w^6*t^8 + 138854464/81*w^6*t^7 + 174246272/81*w^6*t^6 + 137810944/81*w^6*t^5 + 429561856/729*w^6*t^4 - 409600/729*w^6*t^3 + 90800128/243*w^6*t^2 + 552337408/729*w^6*t + 524288*w^6 - 792*w^5*t^14 - 14168*w^5*t^13 - 112224*w^5*t^12 - 1579648/3*w^5*t^11 - 133828352/81*w^5*t^10 - 298406080/81*w^5*t^9 - 47910348

In [16]:
R.<a1,a2,a3,a4,a5,a6>=QQ[]
P.<x,y,z>=ProjectiveSpace(FractionField(R),2)
f=DynamicalSystem([x^2 + a1*y*z + a2*x*z +a3*z^2, a4*x*z + y^2 + a5*y*z + a6*z^2 , z^2])
s=sigma(f,1, chow=False)

Dynamical System of Projective Space of dimension 2 over Fraction Field of Multivariate Polynomial Ring in a1, a2, a3, a4, a5, a6 over Rational Field
  Defn: Defined on coordinates by sending (x : y : z) to
        (x^2 + a2*x*z + a1*y*z + a3*z^2 : y^2 + a4*x*z + a5*y*z + a6*z^2 : z^2) 1
j: 2
im: [v0, v1]
[x0^2 + a2*x0 + a1*x1 + a3, x1^2 + a4*x0 + a5*x1 + a6]
(4, 8)
j: 1
im: [v0, 0]
[(x0^2 + a2*x0*x1 + a3*x1^2 + a1*x1)/(a4*x0*x1 + a6*x1^2 + a5*x1 + 1), x1^2/(a4*x0*x1 + a6*x1^2 + a5*x1 + 1)]
(6, 12)
j: 0
im: [0, 0]
[(x0^2 + a5*x0*x1 + a6*x1^2 + a4*x1)/(a1*x0*x1 + a3*x1^2 + a2*x1 + 1), x1^2/(a1*x0*x1 + a3*x1^2 + a2*x1 + 1)]
(7, 14)


In [17]:
w,t = s.parent().gens()
N=2
D=7
L=[[0 for _ in range(N*D+1)] for _ in range(D+1)]
for i in range(D-1,-1,-1):
    for j in range(N*(D-i)+1):
        if i == 0:
            if j == 0:
                c = R(s.constant_coefficient())
            else:
                c = R(s.subs({w:0}).coefficient({t:j}).subs({w:0,t:0}))
        else:
            c = R(s.coefficient({w:i,t:j}).subs({w:0,t:0}))
        if c.degree()!=0:
            if c.lc() <= 0:
                print D-i,N*(D-i)-j
                c = (-1)*c
            L[D-i][N*(D-i)-j]=c

1 2
2 4
2 3
2 2
3 5
3 4
3 3
4 6
4 2
5 10
5 8
5 7
5 5
5 4
5 3
6 12
6 11
6 10
6 9
6 6
6 2
7 14
7 13
7 12
7 11
7 10
7 8
7 7
7 4
7 3


In [18]:
B=[]
for i in range(len(L)):
    ll = L[i]
    for j in range(len(ll)):
        l = ll[j]
        I = R.ideal(B)
        if l != 0:
            if l not in I:
                print "good: ",(i,j)
                B.append(l)
                print l
            else:
                print "redundant: ",(i,j)
             # print l.lift(I)
len(B)

good:  (1, 2)
8*a1*a4 + 4
good:  (2, 2)
2*a2^2 - 36*a1*a4 + 4*a2*a4 + 4*a1*a5 + 2*a5^2 - 4*a1 - 4*a2 - 8*a3 - 4*a4 - 4*a5 - 8*a6 - 60
good:  (2, 3)
16*a1*a2*a4 + 8*a2^2*a4 + 16*a1*a4*a5 + 8*a1*a5^2 - 4*a2^2 + 8*a1*a4 - 24*a2*a4 - 32*a3*a4 - 24*a1*a5 - 4*a5^2 - 32*a1*a6 + 16*a1 + 8*a2 + 16*a3 + 16*a4 + 8*a5 + 16*a6 + 24
good:  (2, 4)
4*a2^3*a4 - 18*a1^2*a4^2 + 8*a1*a2*a4*a5 + 2*a2^2*a5^2 + 4*a1*a5^3 - 24*a1*a2*a4 - 20*a2^2*a4 - 16*a2*a3*a4 - 4*a2^2*a5 - 24*a1*a4*a5 - 20*a1*a5^2 - 4*a2*a5^2 - 8*a3*a5^2 - 8*a2^2*a6 - 16*a1*a5*a6 + 4*a2^2 + 28*a1*a4 + 32*a2*a4 + 48*a3*a4 + 32*a1*a5 + 8*a2*a5 + 16*a3*a5 + 4*a5^2 + 48*a1*a6 + 16*a2*a6 + 32*a3*a6 - 16*a1 - 8*a2 - 16*a3 - 16*a4 - 8*a5 - 16*a6
redundant:  (3, 2)
good:  (3, 3)
8*a1^2*a4 - 64*a1*a2*a4 - 32*a2^2*a4 + 8*a1*a4^2 - 64*a1*a4*a5 - 32*a1*a5^2 + 32*a2^2 + 128*a2*a4 + 128*a3*a4 + 128*a1*a5 + 32*a5^2 + 128*a1*a6 - 96*a1 - 64*a2 - 128*a3 - 96*a4 - 64*a5 - 128*a6 - 176
redundant:  (3, 4)
redundant:  (3, 5)
redundant:  (3, 6)
redundant:  (4, 

5

In [ ]:
J=R.ideal([B[0]+116, B[1]-40])
J.dimension()

In [33]:
J=R.ideal([B[0]-12, B[1]+116, B[2]-40, B[3]-12, B[4]+352])
J.dimension()

2

In [79]:
g=f.change_ring(R).specialization({a1:1,a2:1,a3:1,a4:1,a5:-1,a6:1})
s=sigma(g,1)

Dynamical System of Projective Space of dimension 2 over Rational Field
  Defn: Defined on coordinates by sending (x : y : z) to
        (x^2 + x*z + y*z + z^2 : y^2 + x*z - y*z + z^2 : z^2) 1
j: 2
im: [v0, v1]
[x0^2 + x0 + x1 + 1, x1^2 + x0 - x1 + 1]
(4, 8)
j: 1
im: [v0, 0]
[(x0^2 + x0*x1 + x1^2 + x1)/(x0*x1 + x1^2 - x1 + 1), x1^2/(x0*x1 + x1^2 - x1 + 1)]
(6, 12)
j: 0
im: [0, 0]
[(x0^2 - x0*x1 + x1^2 + x1)/(x0*x1 + x1^2 + x1 + 1), x1^2/(x0*x1 + x1^2 + x1 + 1)]
(7, 14)


In [89]:
g=f.change_ring(R).specialization({a1:1,a2:1,a3:2,a4:1,a5:5,a6:6})
s2=sigma(g,1)
print s==s2

Dynamical System of Projective Space of dimension 2 over Rational Field
  Defn: Defined on coordinates by sending (x : y : z) to
        (x^2 + x*z + y*z + 2*z^2 : y^2 + x*z + 5*y*z + 6*z^2 : z^2) 1
j: 2
im: [v0, v1]
[x0^2 + x0 + x1 + 2, x1^2 + x0 + 5*x1 + 6]
(4, 8)
j: 1
im: [v0, 0]
[(x0^2 + x0*x1 + 2*x1^2 + x1)/(x0*x1 + 6*x1^2 + 5*x1 + 1), x1^2/(x0*x1 + 6*x1^2 + 5*x1 + 1)]
(6, 12)
j: 0
im: [0, 0]
[(x0^2 + 5*x0*x1 + 6*x1^2 + x1)/(x0*x1 + 2*x1^2 + x1 + 1), x1^2/(x0*x1 + 2*x1^2 + x1 + 1)]
(7, 14)
True
w^7 - 7*w^6*t^2 + 10*w^6*t - 12*w^6 + 21*w^5*t^4 - 60*w^5*t^3 + 116*w^5*t^2 - 40*w^5*t - 12*w^5 - 35*w^4*t^6 + 150*w^4*t^5 - 400*w^4*t^4 + 352*w^4*t^3 + 44*w^4*t^2 - 920*w^4*t + 800*w^4 + 35*w^3*t^8 - 200*w^3*t^7 + 680*w^3*t^6 - 1008*w^3*t^5 + 392*w^3*t^4 + 2496*w^3*t^3 - 3344*w^3*t^2 + 320*w^3*t + 2624*w^3 - 21*w^2*t^10 + 150*w^2*t^9 - 620*w^2*t^8 + 1312*w^2*t^7 - 1320*w^2*t^6 - 1456*w^2*t^5 + 2800*w^2*t^4 + 2336*w^2*t^3 - 10432*w^2*t^2 + 5248*w^2*t + 7*w*t^12 - 60*w*t^11 + 292*w*t^10 - 80

In [122]:
S.<b5,b6>=QQ[]
for c in range(-5,5):
    phi=R.hom([1,2,c,1,b5,b6],S)
    J3=S.ideal([phi(b) for b in J.gens()])
    print c,
    for D in J3.variety():
        print D
        g=f.change_ring(R).specialization({a1:1,a2:1,a3:c,a4:1,a5:D[b5],a6:D[b6]})
        print g.is_morphism()
        s2=sigma(g,1)
        print s==s2

-5 {b6: 481/16, b5: -19/2}
True
Dynamical System of Projective Space of dimension 2 over Rational Field
  Defn: Defined on coordinates by sending (x : y : z) to
        (x^2 + x*z + y*z - 5*z^2 : y^2 + x*z - 19/2*y*z + 481/16*z^2 : z^2) 1
j: 2
im: [v0, v1]
[x0^2 + x0 + x1 - 5, x1^2 + x0 - 19/2*x1 + 481/16]
(4, 8)
j: 1
im: [v0, 0]
[(x0^2 + x0*x1 - 5*x1^2 + x1)/(x0*x1 + 481/16*x1^2 - 19/2*x1 + 1), x1^2/(x0*x1 + 481/16*x1^2 - 19/2*x1 + 1)]
(6, 12)
j: 0
im: [0, 0]
[(x0^2 - 19/2*x0*x1 + 481/16*x1^2 + x1)/(x0*x1 - 5*x1^2 + x1 + 1), x1^2/(x0*x1 - 5*x1^2 + x1 + 1)]
(7, 14)
False
{b6: 849/16, b5: -27/2}
True
Dynamical System of Projective Space of dimension 2 over Rational Field
  Defn: Defined on coordinates by sending (x : y : z) to
        (x^2 + x*z + y*z - 5*z^2 : y^2 + x*z - 27/2*y*z + 849/16*z^2 : z^2) 1
j: 2
im: [v0, v1]
[x0^2 + x0 + x1 - 5, x1^2 + x0 - 27/2*x1 + 849/16]
(4, 8)
j: 1
im: [v0, 0]
[(x0^2 + x0*x1 - 5*x1^2 + x1)/(x0*x1 + 849/16*x1^2 - 27/2*x1 + 1), x1^2/(x0*x1 + 849/16*x1^2 

im: [v0, 0]
[(x0^2 + x0*x1 + 2*x1^2 + x1)/(x0*x1 + 89/16*x1^2 + 9/2*x1 + 1), x1^2/(x0*x1 + 89/16*x1^2 + 9/2*x1 + 1)]
(6, 12)
j: 0
im: [0, 0]
[(x0^2 + 9/2*x0*x1 + 89/16*x1^2 + x1)/(x0*x1 + 2*x1^2 + x1 + 1), x1^2/(x0*x1 + 2*x1^2 + x1 + 1)]
(7, 14)
False
3 {b6: 17/16, b5: 5/2}
True
Dynamical System of Projective Space of dimension 2 over Rational Field
  Defn: Defined on coordinates by sending (x : y : z) to
        (x^2 + x*z + y*z + 3*z^2 : y^2 + x*z + 5/2*y*z + 17/16*z^2 : z^2) 1
j: 2
im: [v0, v1]
[x0^2 + x0 + x1 + 3, x1^2 + x0 + 5/2*x1 + 17/16]
(4, 8)
j: 1
im: [v0, 0]
[(x0^2 + x0*x1 + 3*x1^2 + x1)/(x0*x1 + 17/16*x1^2 + 5/2*x1 + 1), x1^2/(x0*x1 + 17/16*x1^2 + 5/2*x1 + 1)]
(6, 12)
j: 0
im: [0, 0]
[(x0^2 + 5/2*x0*x1 + 17/16*x1^2 + x1)/(x0*x1 + 3*x1^2 + x1 + 1), x1^2/(x0*x1 + 3*x1^2 + x1 + 1)]
(7, 14)
False
{b6: 161/16, b5: 13/2}
True
Dynamical System of Projective Space of dimension 2 over Rational Field
  Defn: Defined on coordinates by sending (x : y : z) to
        (x^2 + x*z + y*z + 

In [113]:
S.<b6,b5,b3,b2>=PolynomialRing(QQ,order='lex')
phi=R.hom([1,b2,b3,1,b5,b6],S)
J3=S.ideal([phi(b) for b in J.gens()])
G=J3.groebner_basis()

In [121]:
for g in G:
    print g.factor()

(1/16) * (16*b6 - 16*b5*b3 + 4*b5*b2^2 - 8*b5*b2 + 12*b5 + 16*b3^2 - 8*b3*b2^2 + 16*b3*b2 - 8*b3 + b2^4 - 4*b2^3 + 6*b2^2 - 12*b2 - 31)
(1/4) * (2*b5 - 4*b3 + b2^2 - 2*b2 - 1) * (2*b5 - 4*b3 + b2^2 - 2*b2 + 7)


In [ ]:
S.<b1,b2,b3,b4,b5,b6>=PolynomialRing(QQ,order='lex')
phi=R.hom([b1,b2,b3,b4,b5,b6],S)
JJ=S.ideal([phi(b) for b in J.gens()])
JJ

In [87]:
J.subs({a1:1,a2:1,a3:2,a4:1,a5:5,a6:6})

Ideal (0, 0, 0, 0, 0) of Multivariate Polynomial Ring in a1, a2, a3, a4, a5, a6 over Rational Field

In [48]:
JJ2=JJ.subs({b1:1,b4:1})
JJ2.groebner_basis()

[b2^2 - 4*b3 + b5^2 - 4*b6 + 6, b2*b3 - 1/2*b2*b5 - 7/4*b2 + 3/4*b3*b5^2 - 3/2*b3*b5 - 3*b3*b6 + 13/4*b3 - 1/64*b5^6 + 3/32*b5^5 + 3/16*b5^4*b6 - 29/64*b5^4 - 3/4*b5^3*b6 + 13/16*b5^3 - 3/4*b5^2*b6^2 + 23/8*b5^2*b6 - 131/64*b5^2 + 3/2*b5*b6^2 - 11/4*b5*b6 + 59/32*b5 + b6^3 - 17/4*b6^2 + 111/16*b6 - 319/64, b2*b5^2 - 2*b2*b5 - 4*b2*b6 + 3*b2 + 4*b3 + 1/4*b5^4 - b5^3 - 2*b5^2*b6 + 3/2*b5^2 + 4*b5*b6 - 3*b5 + 4*b6^2 - 2*b6 - 31/4, b3^2 - 1/8*b3*b5^4 + 1/2*b3*b5^3 + b3*b5^2*b6 - 7/4*b3*b5^2 - 2*b3*b5*b6 + 3/2*b3*b5 - 2*b3*b6^2 + 5*b3*b6 - 49/8*b3 + 1/256*b5^8 - 1/32*b5^7 - 1/16*b5^6*b6 + 11/64*b5^6 + 3/8*b5^5*b6 - 17/32*b5^5 + 3/8*b5^4*b6^2 - 27/16*b5^4*b6 + 163/128*b5^4 - 3/2*b5^3*b6^2 + 15/4*b5^3*b6 - 59/32*b5^3 - b5^2*b6^3 + 21/4*b5^2*b6^2 - 111/16*b5^2*b6 + 219/64*b5^2 + 2*b5*b6^3 - 13/2*b5*b6^2 + 47/8*b5*b6 - 51/32*b5 + b6^4 - 5*b6^3 + 67/8*b6^2 - 149/16*b6 + 1825/256]

In [34]:
J

Ideal (8*a1*a4 - 8, 2*a2^2 - 36*a1*a4 + 4*a2*a4 + 4*a1*a5 + 2*a5^2 - 4*a1 - 4*a2 - 8*a3 - 4*a4 - 4*a5 - 8*a6 + 56, 16*a1*a2*a4 + 8*a2^2*a4 + 16*a1*a4*a5 + 8*a1*a5^2 - 4*a2^2 + 8*a1*a4 - 24*a2*a4 - 32*a3*a4 - 24*a1*a5 - 4*a5^2 - 32*a1*a6 + 16*a1 + 8*a2 + 16*a3 + 16*a4 + 8*a5 + 16*a6 - 16, 4*a2^3*a4 - 18*a1^2*a4^2 + 8*a1*a2*a4*a5 + 2*a2^2*a5^2 + 4*a1*a5^3 - 24*a1*a2*a4 - 20*a2^2*a4 - 16*a2*a3*a4 - 4*a2^2*a5 - 24*a1*a4*a5 - 20*a1*a5^2 - 4*a2*a5^2 - 8*a3*a5^2 - 8*a2^2*a6 - 16*a1*a5*a6 + 4*a2^2 + 28*a1*a4 + 32*a2*a4 + 48*a3*a4 + 32*a1*a5 + 8*a2*a5 + 16*a3*a5 + 4*a5^2 + 48*a1*a6 + 16*a2*a6 + 32*a3*a6 - 16*a1 - 8*a2 - 16*a3 - 16*a4 - 8*a5 - 16*a6 - 12, 8*a1^2*a4 - 64*a1*a2*a4 - 32*a2^2*a4 + 8*a1*a4^2 - 64*a1*a4*a5 - 32*a1*a5^2 + 32*a2^2 + 128*a2*a4 + 128*a3*a4 + 128*a1*a5 + 32*a5^2 + 128*a1*a6 - 96*a1 - 64*a2 - 128*a3 - 96*a4 - 64*a5 - 128*a6 + 176) of Multivariate Polynomial Ring in a1, a2, a3, a4, a5, a6 over Rational Field

In [3]:
#symmetric lattes family from Gauthoer-Hutz-Kaschner
#should have sigmas independent of t
R.<u> = GF(3)[]
P.<x,y,z>=ProjectiveSpace(FractionField(R),2)
u=5
P.<x,y,z>=ProjectiveSpace(GF(11),2)
f=DynamicalSystem([((x+u*z)^2 - u*y^2)^2, 4*((x + u*z)^3*y + 2*(u + 1)*(x + u*z)^2*x*z + u*(x + u*z)*y^3 - 8*u*x*y*z*(x + u*z) - (u + 1)*(x^2*y^2 + u^2*y^2*z^2)), 4*4*x*z*(x-y+z)*(x-u*y+u^2*z)])

In [4]:
sigma(f,1,chow=True)

Dynamical System of Projective Space of dimension 2 over Finite Field of size 11
  Defn: Defined on coordinates by sending (x : y : z) to
        (x^4 + x^2*y^2 + 3*y^4 - 2*x^3*z - x*y^2*z - 4*x^2*z^2 + 3*y^2*z^2 + 5*x*z^3 - 2*z^4 : 4*x^3*y - 2*x^2*y^2 - 2*x*y^3 + 4*x^3*z - x^2*y*z + y^3*z - 4*x^2*z^2 - 5*x*y*z^2 + 5*y^2*z^2 + x*z^3 + 5*y*z^3 : 5*x^3*z + 3*x^2*y*z + 3*x*y^2*z - 2*x^2*z^2 + 4*x*y*z^2 + 4*x*z^3) 1
j: 2
im: [v0, v1]
[(x^4 + x^2*y^2 + 3*y^4 - 2*x^3 - x*y^2 - 4*x^2 + 3*y^2 + 5*x - 2)/(5*x^3 + 3*x^2*y + 3*x*y^2 - 2*x^2 + 4*x*y + 4*x), (4*x^3*y - 2*x^2*y^2 - 2*x*y^3 + 4*x^3 - x^2*y + y^3 - 4*x^2 - 5*x*y + 5*y^2 + x + 5*y)/(5*x^3 + 3*x^2*y + 3*x*y^2 - 2*x^2 + 4*x*y + 4*x)]


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "sage/misc/cachefunc.pyx", line 1943, in sage.misc.cachefunc.CachedMethodCaller.__call__ (build/cythonized/sage/misc/cachefunc.c:10303)
    return cache[k]
KeyError: (('', None, None, False), ())

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/alex/Documents/Sage/sage/local/lib/python3.7/site-packages/sage/libs/singular/standard_options.py", line 141, in wrapper
    return func(*args, **kwds)
  File "/home/alex/Documents/Sage/sage/local/lib/python3.7/site-packages/sage/rings/polynomial/multi_polynomial_ideal.py", line 538, in _groebner_basis_libsingular
    S = groebner(self)
  File "sage/libs/singular/function.pyx", line 1334, in sage.libs.singular.function.SingularFunction.__call__ (build/cythonized/sage/libs/singular/function.cpp:15209)
    return call_function(self, args, ring, interruptible, attributes)
  File "sage/libs/singular/function.pyx", line 1516, in sage.libs.s

TypeError: can only concatenate str (not "list") to str

In [ ]:
a=0;b=1;c=-2
P.<x,y,z>=ProjectiveSpace(QQ,2)
f=DynamicalSystem([x^2 + a*z^2, b*x^2 + c*y^2, z^2])
d=1
f=DynamicalSystem([d*(x^2 + a*z^2), b*y^2 + c*d*x*z, z^2])
d=2
g=DynamicalSystem([d*(x^2 + a*z^2), b*y^2 + c*d*x*z, z^2])
f.conjugating_set(g)

In [ ]:
R.<y>=QQ[]
for c in range(1,20):
    for a in range(-c,c):
        if not (y^2+a*y-c).is_irreducible() and not (y^2+a*y+2*c).is_irreducible():
            print a,c

In [ ]:
a=-2;b=1;e=9+1;c=10
#a=0;b=2;c=-4
P.<x,y,z>=ProjectiveSpace(QQ,2)
f=DynamicalSystem([x^2 + a*z^2, b*x^2 + c*y^2, z^2])
for d in range(1,4):
    f=DynamicalSystem([d*(x^2 + a*z^2), d*(b*y^2 + e*y*z + c*x*z), z^2])
    print d,f.is_morphism(),
    s=sigma(f,1, chow=False)
    if d == 1:
        s1=s
    print s==s1

In [ ]:
R.<s1,s2,t1,t2,S1,S2,T1,T2>=QQ[]
S.<w,t>=PolynomialRing(R, order='lex')
u=(w-(t-s1)*(t-t1))*(w-(t-s1)*(t-t2))*(w-(t-s2)*(t-t1))*(w-(t-s2)*(t-t2))*(w-(t-0)*(t-0))*(w-(t-0)*(t-0))*(w-(t-2)*(t-0))

In [ ]:
u

In [ ]:
N=2
D=7
L=[[0 for _ in range(N*D+1)] for _ in range(D+1)]
for i in range(D-1,-1,-1):
    for j in range(N*(D-i)+1):
        if i == 0:
            if j == 0:
                c = u.constant_coefficient()
            else:
                c = u.subs({w:0}).coefficient({t:j})
        else:
            c = u.coefficient({w:i,t:j})
        if R(c).degree()!=0:
            if R(c).lc() <= 0:
                c = (-1)*c
            L[D-i][N*(D-i)-j]=c
#        print D-i,N*(D-i)-j,c
#        print"--"
#    print "-----"

In [14]:
L[1][2], L[2][2]

(l1*l2*l3 + l1*l2*l4 + l1*l3*l4 + l2*l3*l4 + l1*l2*l5 + l3*l4*l5 + l1*l2*l6 + l3*l4*l6 + l1*l5*l6 + l2*l5*l6 + l3*l5*l6 + l4*l5*l6,
 l1*l2 + l1*l3 + l2*l3 + l1*l4 + l2*l4 + l3*l4 + l1*l5 + l2*l5 + l3*l5 + l4*l5 + l1*l6 + l2*l6 + l3*l6 + l4*l6 + l5*l6)

In [ ]:
J=R.ideal([L[6][1]])
for i in range(D-1,-1,-1):
    for j in range(N*(D-i)+1):
        if L[i][j] != 0 and L[i][j] not in J:
            print i,j
            J = R.ideal(list(J.gens()) + [L[i][j]])

In [ ]:
G = [S1 - (s1+s2), S2 - (s1*s2), T1 - (t1+t2), T2-(t1*t2)]
for i in range(D-1,-1,-1):
    for j in range(N*(D-i)+1):
        if L[i][j] != 0:
            I=R.ideal(G+[L[i][j]])
            J=I.elimination_ideal([s1,s2,t1,t2])
            print i,j,J.gen(0).degrees()
            print J.gen(0)

In [ ]:
I

In [ ]:
R.<a1,a2,a3,a4,a5,a6,b1,b2,b3,b4,b5,b6,c1,c2,c3,c4,c5,c6>=QQ[]
P.<x,y,z>=ProjectiveSpace(R,2)
f=DynamicalSystem([1*x^2+a2*x*y+a3*x*z+0*y^2+a5*y*z + 0*z^2, 0*x^2+b2*x*y+b3*x*z+1*y^2+b5*y*z+0*z^2,z^2])
f(P(0,0,1)), f(P(0,1,0)), f(P(1,0,0))

In [ ]:
f

In [ ]:
R.<l1,l2,l3,l4,l5,l6,l7,l8,l9,l10,l11,l12,l13,l14>=QQ[]
S.<w,t>=PolynomialRing(R, order='lex')
u=(w-(t-l1)*(t-l2))*(w-(t-l3)*(t-l4))*(w-(t-l5)*(t-l6))*(w-(t-l7)*(t-l8))*(w-(t-l9)*(t-l10))*(w-(t-l11)*(t-l12))*(w-(t-l13)*(t-l14))

In [ ]:
R.ngens()

In [ ]:
R.<s10,s11,s12,s20,s21,s22,s23,s24,s30,s31,s32,s33,s34,s35,s36,s40,s41,s42,s43,s44,s45,s46,s47,s48,s50,s51,s52,s53,s54,s55,s56,s57,s58,s59,s510,s60,s61,s62,s63,s64,s65,s66,s67,s68,s69,s610,s611,s612,s70,s71,s72,s73,s74,s75,s76,s77,s78,s79,s710,s711,s712,s713,s714>=QQ[]
S.<w,t>=PolynomialRing(R, order='lex')
u=0
k=0
for i in range(7,-1,-1):
    if i == 7:
        u = w**7
    else:
        for j in range(14-2*i,-1,-1):
            print i,j,k, R.gen(k)
            u += w**i*t**j*R.gen(k)
            k += 1

In [ ]:
u.discriminant(w)

In [ ]:
u.discriminant(w)

In [ ]:
S.<z>=QQ[]
F=123*z^6+123*z-1123*z^3
f=DynamicalSystem_affine([z - F/F.derivative()]).homogenize(1)
f.sigma_invariants(1)

In [ ]:
#newton
R.<x,y>=QQ[]
F=x^3-y-2
G=y^3+2*x+11
v=vector([F,G])
f=DynamicalSystem_affine(list(vector([x,y])-jacobian(v,[x,y]).inverse()*v))
fp=f.homogenize(2)
fp

In [ ]:
Fix=fp.periodic_points(1, return_scheme=True)

In [ ]:
s=sigma(fp,1)

In [ ]:
s

In [ ]:
s

In [ ]:
#symmetric lattes family from Gauthoer-Hutz-Kaschner
#should have sigmas independent of t
R.<t> = GF(3)[]
P.<x,y,z>=ProjectiveSpace(FractionField(R),2)
t=3
P.<x,y,z>=ProjectiveSpace(GF(7),2)
f=DynamicalSystem([((x+t*z)^2 - t*y^2)^2, 4*((x + t*z)^3*y + 2*(t + 1)*(x + t*z)^2*x*z + t*(x + t*z)*y^3 - 8*t*x*y*z*(x + t*z) - (t + 1)*(x^2*y^2 + t^2*y^2*z^2)), 16*x*z*(x-y+z)*(x-t*y+t^2*z)])

In [ ]:
sigma(f,1, chow=True)

In [ ]:
set_gap_memory_pool_size(32212254720) #30 Gb

In [ ]:
SD.gens()

In [ ]:
#create matrix group

N=2
D=4
SN = SymmetricGroup(N)
SD = SymmetricGroup(D)
SND = SymmetricGroup(N*D)
R = PolynomialRing(QQ,N*D, 'l')
R.inject_variables()
G=[]
vars = [[R.gen(i+N*j) for i in range(N)] for j in range(D)]
d = [1..D]
n = [1..N]
num = 0
for sd in SD:
    new_vars = [vars[i-1] for i in sd(d)]
    for sn1 in SN:
        for sn2 in SN:
            for sn3 in SN:
                for sn4 in SN:
                                sn = [sn1(n),sn2(n),sn3(n),sn4(n)]
                                new_vars = [[new_vars[j][i-1] for i in sn[j]] for j in range(D)]
                                #print h,g, new_vars
                                phi = R.hom(new_vars,R)
                                #print phi
                                L=[R.gens().index(t)+1 for t in phi.im_gens()]
                                G.append(SND(L))

M=MatrixGroup([g.matrix() for g in G])

In [ ]:
M.cardinality()

In [ ]:
#create matrix group

N=2
D=4
SN = SymmetricGroup(N)
SD = SymmetricGroup(D)
SND = SymmetricGroup(N*D)
R = PolynomialRing(QQ,N*D, 'l')
R.inject_variables()
G=[]
vars = [[R.gen(i+N*j) for i in range(N)] for j in range(D)]
d = [1..D]
n = [1..N]
num = 0
for sd in SD.gens():
    new_vars = [vars[i-1] for i in sd(d)]
    for sn1 in SN.gens():
        for sn2 in SN.gens():
            for sn3 in SN.gens():
                for sn4 in SN.gens():
                                sn = [sn1(n),sn2(n),sn3(n),sn4(n)]
                                new_vars = [[new_vars[j][i-1] for i in sn[j]] for j in range(D)]
                                #print h,g, new_vars
                                phi = R.hom(new_vars,R)
                                #print phi
                                L=[R.gens().index(t)+1 for t in phi.im_gens()]
                                G.append(SND(L))
                                print sn, sd, SND(L)
                                print SND(L).matrix()

M=MatrixGroup([g.matrix() for g in G])

In [ ]:
M.cardinality()

In [ ]:
#create matrix group

N=2
D=7
SN = SymmetricGroup(N)
SD = SymmetricGroup(D)
SND = SymmetricGroup(N*D)
R = PolynomialRing(QQ,N*D, 'l')
R.inject_variables()
G=[]
vars = [[R.gen(i+N*j) for i in range(N)] for j in range(D)]
d = [1..D]
n = [1..N]
num = 0
for sd in SD:
    new_vars = [vars[i-1] for i in sd(d)]
    for sn1 in SN:
        for sn2 in SN:
            for sn3 in SN:
                for sn4 in SN:
                    for sn5 in SN:
                        for sn6 in SN:
                            for sn7 in SN:
                                sn = [sn1(n),sn2(n),sn3(n),sn4(n),sn5(n),sn6(n),sn7(n)]
                                new_vars = [[new_vars[j][i-1] for i in sn[j]] for j in range(D)]
                                #print h,g, new_vars
                                phi = R.hom(new_vars,R)
                                #print phi
                                L=[R.gens().index(t)+1 for t in phi.im_gens()]
                                G.append(SND(L))

M=MatrixGroup([g.matrix() for g in G])

In [ ]:
len(M)

In [ ]:
N=2
D=3
SN = SymmetricGroup(N)
SD = SymmetricGroup(D)
SND = SymmetricGroup(N*D)
R.<l1,l2,l3,l4,l5,l6> = PolynomialRing(QQ,N*D)
R.inject_variables()
G=[]
vars = [[R.gen(i+N*j) for i in range(N)] for j in range(D)]
d = [1..D]
n = [1..N]
num = 0
for sd in SD:
    new_vars = [vars[i-1] for i in sd(d)]
    for sn1 in SN:
        for sn2 in SN:
            for sn3 in SN:
                    sn = [sn1(n),sn2(n),sn3(n)]
                    new_vars = [[new_vars[j][i-1] for i in sn[j]] for j in range(D)]
                    #print h,g, new_vars
                    phi = R.hom(new_vars,R)
                    #print phi
                    L=[R.gens().index(t)+1 for t in phi.im_gens()]
                    G.append(SND(L))

M=MatrixGroup([g.matrix() for g in G])

IN=M.invariant_generators()

In [ ]:
len(IN)

In [ ]:
R.<l1,l2,l3,l4,l5,l6>=QQ[]
S.<w,t>=PolynomialRing(R, order='lex')
u=(w-(t-l1)*(t-l2)*(t-l3))*(w-(t-l4)*(t-l5)*(t-l6))

In [ ]:
R.<l1,l2,l3,l4,l5,l6,l7,l8,l9>=QQ[]
S.<w,t>=PolynomialRing(R, order='lex')
u=(w-(t-l1)*(t-l2)*(t-l3))*(w-(t-l4)*(t-l5)*(t-l6))*(w-(t-l7)*(t-l8)*(t-l9))

In [15]:
R.<l1,l2,l3,l4,l5,l6,l7,l8>=QQ[]
S.<w,t>=PolynomialRing(R, order='lex')
u=(w-(t-l1)*(t-l2))*(w-(t-l3)*(t-l4))*(w-(t-l5)*(t-l6))*(w-(t-l7)*(t-l8))

In [ ]:
R.<l1,l2,l3,l4,l5,l6,l7,l8,l9,l10>=QQ[]
S.<w,t>=PolynomialRing(R, order='lex')
u=(w-(t-l1)*(t-l2))*(w-(t-l3)*(t-l4))*(w-(t-l5)*(t-l6))*(w-(t-l7)*(t-l8))*(w-(t-l9)*(t-l10))

In [2]:
R.<l1,l2,l3,l4,l5,l6>=QQ[]
S.<w,t>=PolynomialRing(R, order='lex')
u=(w-(t-l1)*(t-l2))*(w-(t-l3)*(t-l4))*(w-(t-l5)*(t-l6))

In [ ]:
R.<l1,l2,l3,l4,l5,l6,l7,l8,l9,l10,l11,l12>=QQ[]
S.<w,t>=PolynomialRing(R, order='lex')
u=(w-(t-l1)*(t-l2)*(t-l3))*(w-(t-l4)*(t-l5)*(t-l6))*(w-(t-l7)*(t-l8)*(t-l9))*(w-(t-l10)*(t-l11)*(t-l12))

In [ ]:
v1=u.subs({w:t^2+1})
v1.coefficients()
v2=u.subs({w:t^2})
v2.coefficients()
v3=u.subs({w:t^2+t})
v3.coefficients()

In [ ]:
I=R.ideal(v1.coefficients() + v2.coefficients() + v3.coefficients())

In [ ]:
R

In [ ]:
I.dimension()

In [ ]:
I.variety()

In [16]:
N=2
D=4
L=[[0 for _ in range(N*D+1)] for _ in range(D+1)]
for i in range(D-1,-1,-1):
    for j in range(N*(D-i)):
        if i == 0:
            if j == 0:
                c = u.constant_coefficient()
            else:
                c = u.subs({w:0}).coefficient({t:j})
        else:
            c = u.coefficient({w:i,t:j})
        if R(c).degree()!=0:
            if R(c).lc() <= 0:
                c = (-1)*c
            L[D-i][N*(D-i)-j]=c
#        print D-i,N*(D-i)-j,c
#        print"--"
#    print "-----"

In [ ]:
l7

In [ ]:
u = (1-l1)*(1-l2)*(1-l3)*(1-l4)*(1-l5)*(1-l6)
#u = (1-l1)*(1-l2)*(1-l3)*(1-l4)*(1-l5)*(1-l6)*(1-l7)*(1-l8)
u1 = (1-l1)*(1-l2)
u2 = (1-l3)*(1-l4)
u3 = (1-l5)*(1-l6)
u4 = (1-l7)*(1-l8)

u*((l1*l2)/u1 + (l3*l4)/u2 + (l5*l6)/u3) - 2*u

In [ ]:
u = (1-l1)*(1-l2)*(1-l3)*(1-l4)*(1-l5)*(1-l6)*(1-l7)*(1-l8)*(1-l9)
u1 = (1-l1)*(1-l2)*(1-l3)
u2 = (1-l4)*(1-l5)*(1-l6)
u3 = (1-l7)*(1-l8)*(1-l9)

ww=u*((l1+l2+l3)/u1 + (l4+l5+l6)/u2 + (l7+l8+l9)/u3) - 2*u

In [ ]:
u = (1-l1)*(1-l2)*(1-l3)*(1-l4)*(1-l5)*(1-l6)*(1-l7)
ww=1/(1-l1) + 1/(1-l2) + 1/(1-l3) + 1/(1-l4) + 1/(1-l5) + 1/(1-l6) + 1/(1-l7)
u*ww

In [17]:
B=[]
for i in range(len(L)):
    ll = L[i]
    for j in range(len(ll)):
        l = ll[j]
        I = R.ideal(B)
        if l != 0:
            if l not in I:
                print "good: ",(i,j)
                B.append(l)
            else:
                print "redundant: ",(i,j)
             # print l.lift(I)
len(B)

good:  (1, 1)
good:  (1, 2)
redundant:  (2, 1)
good:  (2, 2)
good:  (2, 3)
good:  (2, 4)
redundant:  (3, 1)
redundant:  (3, 2)
good:  (3, 3)
good:  (3, 4)
good:  (3, 5)
good:  (3, 6)
redundant:  (4, 1)
redundant:  (4, 2)
redundant:  (4, 3)
good:  (4, 4)
good:  (4, 5)
good:  (4, 6)
good:  (4, 7)
good:  (4, 8)


14

In [22]:
L[1][2], L[2][2], L[3][2]

(l1*l2 + l3*l4 + l5*l6 + l7*l8,
 3*l1*l2 + l1*l3 + l2*l3 + l1*l4 + l2*l4 + 3*l3*l4 + l1*l5 + l2*l5 + l3*l5 + l4*l5 + l1*l6 + l2*l6 + l3*l6 + l4*l6 + 3*l5*l6 + l1*l7 + l2*l7 + l3*l7 + l4*l7 + l5*l7 + l6*l7 + l1*l8 + l2*l8 + l3*l8 + l4*l8 + l5*l8 + l6*l8 + 3*l7*l8,
 3*l1*l2 + 2*l1*l3 + 2*l2*l3 + 2*l1*l4 + 2*l2*l4 + 3*l3*l4 + 2*l1*l5 + 2*l2*l5 + 2*l3*l5 + 2*l4*l5 + 2*l1*l6 + 2*l2*l6 + 2*l3*l6 + 2*l4*l6 + 3*l5*l6 + 2*l1*l7 + 2*l2*l7 + 2*l3*l7 + 2*l4*l7 + 2*l5*l7 + 2*l6*l7 + 2*l1*l8 + 2*l2*l8 + 2*l3*l8 + 2*l4*l8 + 2*l5*l8 + 2*l6*l8 + 3*l7*l8)

In [ ]:
B=[]
for i in range(len(L)):
    ll = L[i]
    for j in range(len(ll)):
        b = len(ll)-1
        l = ll[b-j]
        I = R.ideal(B)
        if l != 0:
            if l not in I:
                print "good: ",(i,b-j)
                B.append(l)
            else:
                print "redundant: ",(i,b-j)
             # print l.lift(I)
len(B)

In [ ]:
I=R.ideal([R(L[1][2]),R(L[2][2])])

In [ ]:
R(L[3][2]).lift(I)

In [ ]:
e = SymmetricFunctions(QQ).elementary()
ll=[[l1,l2],[l3,l4],[l5,l6]]
a1=e[2].expand(N)(ll[0])*e[1].expand(N)(ll[1])+e[2].expand(N)(ll[0])*e[1].expand(N)(ll[2])+e[2].expand(N)(ll[1])*e[1].expand(N)(ll[2])
a2=e[1].expand(N)(ll[0])*e[2].expand(N)(ll[1])+e[1].expand(N)(ll[0])*e[2].expand(N)(ll[2])+e[1].expand(N)(ll[1])*e[2].expand(N)(ll[2])

In [ ]:
(s.numerator() - s.denominator()) in I

In [ ]:
list(Partitions(3,parts_in=[1..2]))

In [ ]:
p= Partitions(4,parts_in=[1..2])
p.cardinality()

In [ ]:
ind=0
D=5
N=3
for k in range(1,N*D+1):
    p= Partitions(k,max_part=N, max_length=D)
    print list(p), k
    ind += p.cardinality()
print ind

In [ ]:
B=[]
for l in [L[k][3] for k in range(1,5)]:
        I = R.ideal(B)
        if l not in I:
            B.append(l)
        else:
            print "redundant ", l
         # print l.lift(I)
B

In [ ]:
len(B)

In [ ]:
B

In [ ]:
B=[]
for ll in L:
    for l in ll:
        I = R.ideal(B)
        if l not in I:
            B.append(l)
        else:
            print "redundant ", l
         # print l.lift(I)
B

In [ ]:
deg=[R(b).degree() for b in B]
sorted(deg)

In [ ]:
e = SymmetricFunctions(QQ).elementary()
ll=[[l1,l2],[l3,l4],[l5,l6]]
a1=e[2].expand(N)(ll[0])*e[1].expand(N)(ll[1])+e[2].expand(N)(ll[0])*e[1].expand(N)(ll[2])+e[2].expand(N)(ll[1])*e[1].expand(N)(ll[2])
a2=e[1].expand(N)(ll[0])*e[2].expand(N)(ll[1])+e[1].expand(N)(ll[0])*e[2].expand(N)(ll[2])+e[1].expand(N)(ll[1])*e[2].expand(N)(ll[2])

In [ ]:
ll=[[l1,l2],[l3,l4],[l5,l6],[l7,l8]]
a1=e[2].expand(N)(ll[0])*e[1].expand(N)(ll[1])+e[2].expand(N)(ll[0])*e[1].expand(N)(ll[2])+e[2].expand(N)(ll[1])*e[1].expand(N)(ll[2])

In [ ]:
a1+a2

In [ ]:
e[1].expand(N)(ll[0])*e[1].expand(N)(ll[1])*e[1].expand(N)(ll[2])

In [ ]:
L[2][3]+L[3][3]

In [ ]:
-(3,6) + (3,5) -(2,4) - (3,4) + (3,3) + (2,3) -(3,2) - (2,2) + (1,2) - (1,1) + 2

In [ ]:
L[4][3]+L[3][3]+ L[2][3]== -L[1][3]

In [ ]:
phi(L[2][2])

In [ ]:
S.<l1,l2,l3,l4,l5,l6,l7,l8,l9,l10,l11,l12,u1,u2,u3> = PolynomialRing(QQ,15,order='lex')
phi=R.hom([l1,l2,l3,l4,l5,l6,l7,l8,l9,l10,l11,l12],S)
I=S.ideal([u1-phi(L[4][4]),u2-phi(L[3][4]),u3-phi(L[2][4])])

In [ ]:
G=I.groebner_basis()
G[-1]

In [ ]:
r.<l1,l2,l3,l4,l5,l6,l7,l8,l9>=QQ[]
S=0
#for i in range(0,6,2):
#    S+=1/((1-r.gen(i))*(1-r.gen(i+1)))
for i in range(0,8,3):
    S+=1/((1-r.gen(i))*(1-r.gen(i+1))*(1-r.gen(i+2)))
S.numerator() - S.denominator()

In [ ]:
e = SymmetricFunctions(QQ).elementary()
ll=[[l1,l2,l3],[l4,l5,l6],[l7,l8,l9]]

In [ ]:
a1=e[3].expand(D)(ll[0])*e[2].expand(D)(ll[1])+e[3].expand(D)(ll[0])*e[2].expand(D)(ll[2])+e[3].expand(D)(ll[1])*e[2].expand(D)(ll[2])
a2=e[2].expand(D)(ll[0])*e[3].expand(D)(ll[1])+e[2].expand(D)(ll[0])*e[3].expand(D)(ll[2])+e[2].expand(D)(ll[1])*e[3].expand(D)(ll[2])

In [ ]:
L[7]

In [ ]:
L[16]

In [ ]:
L[4]

In [ ]:
a1=e[3].expand(D)(ll[0])*e[1].expand(D)(ll[1])+e[3].expand(D)(ll[0])*e[1].expand(D)(ll[2])+e[3].expand(D)(ll[1])*e[1].expand(D)(ll[2])
a2=e[1].expand(D)(ll[0])*e[3].expand(D)(ll[1])+e[1].expand(D)(ll[0])*e[3].expand(D)(ll[2])+e[1].expand(D)(ll[1])*e[3].expand(D)(ll[2])
a3=e[2].expand(D)(ll[0])*e[2].expand(D)(ll[1])+e[2].expand(D)(ll[0])*e[2].expand(D)(ll[2])+e[2].expand(D)(ll[1])*e[2].expand(D)(ll[2])

In [ ]:
a1=e[2].expand(D)(ll[0])+e[2].expand(D)(ll[1])+e[2].expand(D)(ll[2])
a2=e[1].expand(D)(ll[0])*e[1].expand(D)(ll[1])+e[1].expand(D)(ll[0])*e[1].expand(D)(ll[2])+e[1].expand(D)(ll[1])*e[1].expand(D)(ll[2])

In [ ]:
a1+a2

In [ ]:
L[-2]

In [ ]:
L[5]==a1+a2+a3

In [ ]:
L[5]

In [ ]:
L=[]
for i in range(R.ngens()/2):
    if i == 0:
        z = u.subs({w:0})
        #print z
        for v in z.coefficients():
            if v.degree()!=0:
                L.append(v)
            print v
            print "--"
    else:
        #print u.coefficient(w**i)
        for v in u.coefficient(w**i).coefficients():
            print v
            if v.degree()!=0:
                L.append(v)
            print"--"
    print "-----"

In [ ]:
L

In [ ]:
B=[]
for l in L:
    I = R.ideal(B)
    if l not in I:
        B.append(l)
    else:
        print "redundant ", l
        #print l.lift(I)
B

In [ ]:
invariant_generators(M)

In [ ]:
for m in M:
    s=[b for a in m.matrix() for b in a]
    print s

In [ ]:
e = SymmetricFunctions(QQ).elementary()
p = SymmetricFunctions(QQ).power()

In [ ]:
R.<l1,l2,l3,l4,l5,l6>=QQ[]
L1=[]; L2=[]; L3=[]
L1.append((l1+l2))
L1.append((l1*l2))
L2.append((l3+l4))
L2.append((l3*l4))
L3.append((l5+l6))
L3.append((l5*l6))

In [ ]:
R.<l1,l2,l3,l4,l5,l6>=QQ[]
e = SymmetricFunctions(QQ).elementary()
D=2
N=3
IN=[]
L=[[l1,l2],[l3,l4],[l5,l6]]
d = {l1:l1+l2, l2:l1*l3, l3:l3+l4,l4:l3*l4, l5:l5+l6, l6:l5*l6}
print (l1+l2).subs(d)
for i in range(1,N+1):
    for j in range(1,D+1):
        w=e[i].expand(N)([e[j].expand(D)(L[k]) for k in range(N)])
        print i,j,w
        IN.append(w)

In [ ]:
D=2
N=3
IN=[]
L=[[l1,l2],[l3,l4],[l5,l6]]
for i in range(1,N+1):
    for j in range(1,D+1):
        w=e[i].expand(N)([e[j].expand(D)(L[k]) for k in range(N)])
        print i,j,w
        IN.append(w)

In [ ]:
IN

In [ ]:
u=l1*l2*l3*l4*l5 + l1*l2*l3*l4*l6 + l1*l2*l3*l5*l6 + l1*l2*l4*l5*l6 + l1*l3*l4*l5*l6 + l2*l3*l4*l5*l6
I=R.ideal(IN)
u in I

In [ ]:
M.invariant_generators()

In [ ]:
I

In [ ]:
# does not get them all!
for a in L1:
    for b in L2:
        for c in L3:
            L=[a,b,c]
            L,e[1].expand(3)(L), e[2].expand(3)(L), e[3].expand(3)(L)

In [ ]:
R.<l1,l2,l3,l4,l5,l6,l7,l8>=QQ[]
S.<w,t>=PolynomialRing(R, order='lex')
u=(w-(t-l1)*(t-l2))*(w-(t-l3)*(t-l4))*(w-(t-l5)*(t-l6))*(w-(t-l7)*(t-l8))

In [ ]:
L=[]
for i in range(R.ngens()/2):
    if i == 0:
        z = u.subs({w:0})
        print z
        for v in z.coefficients():
            if v.degree()!=0:
                L.append(v)
            print v
            print "--"
    else:
        print u.coefficient(w**i)
        for v in u.coefficient(w**i).coefficients():
            print v
            if v.degree()!=0:
                L.append(v)
            print"--"
    print "-----"

In [ ]:
L.insert(0,L[-1])

In [ ]:
B=[]
for l in L:
    I = R.ideal(B)
    if l not in I:
        B.append(l)
    else:
        print "redundant ", l
        print l.lift(I)
B

In [ ]:
len(B), len(L)

In [ ]:
IN

In [ ]:
I.dimension()

In [ ]:
set_verbose(None)
I=R.ideal(L)
for g in IN:
    R(g) in I
    #R(g).lift(I)

In [ ]:
LT=[]
LT.append(l1*l2*l3*l4*l5*l6)
LT.append(l1*l2*l3*l4*l5 + l1*l2*l3*l4*l6 + l1*l2*l3*l5*l6 + l1*l2*l4*l5*l6 + l1*l3*l4*l5*l6 + l2*l3*l4*l5*l6)
LT.append(l1*l2*l3*l5 + l1*l2*l4*l5 + l1*l3*l4*l5 + l2*l3*l4*l5 + l1*l2*l3*l6 + l1*l2*l4*l6 + l1*l3*l4*l6 + l2*l3*l4*l6 + l1*l3*l5*l6 + l2*l3*l5*l6 + l1*l4*l5*l6 + l2*l4*l5*l6)
LT.append(l1*l3*l5 + l2*l3*l5 + l1*l4*l5 + l2*l4*l5 + l1*l3*l6 + l2*l3*l6 + l1*l4*l6 + l2*l4*l6)
LT.append(l1*l2 + l3*l4 + l5*l6)
LT.append(l1 + l2 + l3 + l4 + l5 + l6)

In [ ]:
set_verbose(None)
I=R.ideal(L)
for g in LT:
    R(g) in I
    R(g).lift(I)

In [ ]:
set_verbose(None)
I=R.ideal(LT)
for g in L:
    R(g) in I
    #R(g).lift(I)

In [ ]:
LT

In [ ]:
L

In [ ]:
#def wreath(d,N,D):
N=2
D=4
SND = SymmetricGroup(N*D)
SN = SymmetricGroup(N)
SD = SymmetricGroup(D)
#R.<l1,l2,l3,l4,l5,l6> = PolynomialRing(QQ,N*D)
#R = PolynomialRing(QQ,N*D, 'l')
#R.inject_variables()
#f = l1*l3*l5 + l2*l3*l5 + l1*l4*l5 + l2*l4*l5 + l1*l3*l6 + l2*l3*l6 + l1*l4*l6 + l2*l4*l6
#f=-(- l1*l2*l3*l5 - l1*l2*l4*l5 - l1*l3*l4*l5 - l2*l3*l4*l5 - l1*l2*l3*l6 - l2*l3*l4*l6 - l1*l3*l5*l6 - l2*l3*l5*l6 - l1*l4*l5*l6 -l2*l4*l5*l6)
G=[]
vars = [[R.gen(i+N*j) for i in range(N)] for j in range(D)]
d = [1..D]
n = [1..N]
num = 0
for sd in SD:
    new_vars = [vars[i-1] for i in sd(d)]
    for sn1 in SN:
        for sn2 in SN:
            for sn3 in SN:
                for sn4 in SN:
                    sn = [sn1(n),sn2(n),sn3(n),sn4(n)]
                    new_vars = [[new_vars[j][i-1] for i in sn[j]] for j in range(D)]
                    #print h,g, new_vars
                    phi = R.hom(new_vars,R)
                    #print phi
                    L=[R.gens().index(t)+1 for t in phi.im_gens()]
                    G.append(SND(L))

In [ ]:
M=MatrixGroup([g.matrix() for g in G])

In [ ]:
M.cardinality()

In [ ]:
IN=M.invariant_generators()

In [ ]:
set_verbose(None)
I=R.ideal(L)
for g in IN:
    R(g) in I
    #R(g).lift(I)

In [ ]:
M.invariants_of_degree(2)

In [ ]:
((x1 + x2 + x3 + x4 + x5 + x0)^2 - (x0^2 + x1^2 + x2^2 + x3^2 + x4^2 + x5^2))/2 - (x0*x1 + x2*x3 + x4*x5)

In [ ]:
G=M.gap()
G.StructureDescription()

In [ ]:
(N,d): structure
(2,3): C2 x S4
(2,4): ((((C2 x D8) : C2) : C3) : C2) : C2
(3,3):

In [ ]:
l1*l2 + l1*l3 + l2*l3 + l1*l4 + l2*l4 + l3*l4 + l1*l5 + l2*l5+ l3*l5 + l4*l5 + l1*l6 + l2*l6 + l3*l6 + l4*l6 + l5*l6

In [ ]:
((x1 + x2 + x3 + x4 + x5 + x0)^2 - (x0^2 + x1^2 + x2^2 + x3^2 + x4^2 + x5^2))/2

In [ ]:
M.invariant_generators()

In [ ]:
e = SymmetricFunctions(QQ).elementary()

In [ ]:
L1=[]
L2=[]
L1.append((l1+l2))
L2.append((l1*l2))
L1.append((l3+l4))
L2.append((l3*l4))
L1.append((l5+l6))
L2.append((l5*l6))

In [ ]:
e[1].expand(3)(L1), e[2].expand(3)(L1), e[3].expand(3)(L1)

In [ ]:
g.matrix()

In [ ]:
#def wreath(d,N,D):
N=2
D=3
SN = SymmetricGroup(N)
SD = SymmetricGroup(D)
R.<l1,l2,l3,l4,l5,l6> = PolynomialRing(QQ,N*D)
R.inject_variables()
f = l1*l3*l5 + l2*l3*l5 + l1*l4*l5 + l2*l4*l5 + l1*l3*l6 + l2*l3*l6 + l1*l4*l6 + l2*l4*l6
f=-(- l1*l2*l3*l5 - l1*l2*l4*l5 - l1*l3*l4*l5 - l2*l3*l4*l5 - l1*l2*l3*l6 - l2*l3*l4*l6 - l1*l3*l5*l6 - l2*l3*l5*l6 - l1*l4*l5*l6 -l2*l4*l5*l6)
f=l1*l2 + l1*l3 + l2*l3 + l1*l4 + l2*l4 + l3*l4 + l1*l5 + l2*l5+ l3*l5 + l4*l5 + l1*l6 + l2*l6 + l3*l6 + l4*l6 + l5*l6
g = 0
print f
vars = [[R.gen(i+N*j) for i in range(N)] for j in range(D)]
d = [1..D]
n = [1..N]
num = 0
for sd in SD:
    new_vars = [vars[i-1] for i in sd(d)]
    for sn in SN:
        new_vars = [[t[i-1] for i in sn(n)] for t in new_vars]
        #print h,g, new_vars
        phi = R.hom(new_vars,R)
        #print phi
        g += phi(f)
        num += 1
g*(5/60)

In [ ]:
(l1*l2*l3*l4*l5*l6 - l1*l2*l3*l4*l5 - l1*l2*l3*l4*l6 - l1*l2*l3*l5*l6
- l1*l2*l4*l5*l6 - l1*l3*l4*l5*l6 - l2*l3*l4*l5*l6 + l1*l2*l3*l4 +
l1*l2*l3*l5 + l1*l2*l4*l5 + l1*l3*l4*l5 + l2*l3*l4*l5 + l1*l2*l3*l6 +
l1*l2*l4*l6 + l1*l3*l4*l6 + l2*l3*l4*l6 + l1*l2*l5*l6 + l1*l3*l5*l6 +
l2*l3*l5*l6 + l1*l4*l5*l6 + l2*l4*l5*l6 + l3*l4*l5*l6 - l1*l2*l3 -
l1*l2*l4 - l1*l3*l4 - l2*l3*l4 - l1*l2*l5 - l1*l3*l5 - l2*l3*l5 -
l1*l4*l5 - l2*l4*l5 - l3*l4*l5 - l1*l2*l6 - l1*l3*l6 - l2*l3*l6 -
l1*l4*l6 - l2*l4*l6 - l3*l4*l6 - l1*l5*l6 - l2*l5*l6 - l3*l5*l6 -
l4*l5*l6 + l1*l2 + l1*l3 + l2*l3 + l1*l4 + l2*l4 + l3*l4 + l1*l5 + l2*l5
+ l3*l5 + l4*l5 + l1*l6 + l2*l6 + l3*l6 + l4*l6 + l5*l6 - l1 - l2 - l3 -
l4 - l5 - l6 + 1)

In [ ]:
N=2
D=3
R.<l1,l2,l3,l4,l5,l6> = PolynomialRing(QQ,N*D)

In [ ]:
L=[]
L.append((l1+l2)*(l3+l4)*(l5+l6))
L.append((l1+l2)*(l3+l4) + (l1+l2)*(l5+l6) + (l3+l4)*(l5+l6))
L.append(l1+l2 + l3+l4 + l5+l6)
L.append(l1*l2 * l3*l4 * l5*l6)
L.append(l1*l2*l3*l4 + l1*l2*l5*l6 + l3*l4*l5*l6)
L.append(l1*l2 + l3*l4 + l5*l6)

In [ ]:
T=-l1*l2*l3*l4*l5*l6 + l1*l2*l3*l4*l5 + l1*l2*l3*l4*l6 + l1*l2*l3*l5*l6 + l1*l2*l4*l5*l6 + l1*l3*l4*l5*l6 + l2*l3*l4*l5*l6 - l1*l2*l3*l5 - l1*l2*l4*l5 - l1*l3*l4*l5 - l2*l3*l4*l5 - l1*l2*l3*l6 - l1*l2*l4*l6 - l1*l3*l4*l6 - l2*l3*l4*l6 - l1*l3*l5*l6 - l2*l3*l5*l6 - l1*l4*l5*l6 - l2*l4*l5*l6 + l1*l3*l5 + l2*l3*l5 + l1*l4*l5 + l2*l4*l5 + l1*l3*l6 + l2*l3*l6 + l1*l4*l6 + l2*l4*l6 + l1*l2 + l3*l4 + l5*l6 - l1 - l2 - l3 - l4 - l5 - l6 + 2
T2 = (l1*l2*l3*l4*l5*l6 - l1*l2*l3*l4*l5 - l1*l2*l3*l4*l6 - l1*l2*l3*l5*l6- l1*l2*l4*l5*l6 - l1*l3*l4*l5*l6 - l2*l3*l4*l5*l6 + l1*l2*l3*l4 +l1*l2*l3*l5 + l1*l2*l4*l5 + l1*l3*l4*l5 + l2*l3*l4*l5 + l1*l2*l3*l6 +l1*l2*l4*l6 + l1*l3*l4*l6 + l2*l3*l4*l6 + l1*l2*l5*l6 + l1*l3*l5*l6 +l2*l3*l5*l6 + l1*l4*l5*l6 + l2*l4*l5*l6 + l3*l4*l5*l6 - l1*l2*l3 -l1*l2*l4 - l1*l3*l4 - l2*l3*l4 - l1*l2*l5 - l1*l3*l5 - l2*l3*l5 -l1*l4*l5 - l2*l4*l5 - l3*l4*l5 - l1*l2*l6 - l1*l3*l6 - l2*l3*l6 -l1*l4*l6 - l2*l4*l6 - l3*l4*l6 - l1*l5*l6 - l2*l5*l6 - l3*l5*l6 -l4*l5*l6 + l1*l2 + l1*l3 + l2*l3 + l1*l4 + l2*l4 + l3*l4 + l1*l5 + l2*l5+ l3*l5 + l4*l5 + l1*l6 + l2*l6 + l3*l6 + l4*l6 + l5*l6 - l1 - l2 - l3 -l4 - l5 - l6 + 1)

In [ ]:
T3=l1*l2 + l1*l3 + l2*l3 + l1*l4 + l2*l4 + l3*l4 + l1*l5 + l2*l5+ l3*l5 + l4*l5 + l1*l6 + l2*l6 + l3*l6 + l4*l6 + l5*l6

In [ ]:
M.invariants_of_degree(1)

In [ ]:
M.invariants_of_degree(2)

In [ ]:
M.invariants_of_degree(3)

In [ ]:
M.invariant_generators()

In [ ]:
M.gens()

In [ ]:
S=R.quotient(L)
S=R.quotient([R(t) for t in M.invariants_of_degree(1)] + [R(t) for t in M.invariants_of_degree(2)])
S=R.quotient([R(t) for t in M.invariant_generators()])
#S=R.quotient([e[1].expand(3)(L1), e[2].expand(3)(L1), e[3].expand(3)(L1),e[1].expand(3)(L2), e[2].expand(3)(L2), e[3].expand(3)(L2)])
S(T).lift()

In [ ]:
T2

In [ ]:
LT=[]
LT.append(l1*l2*l3*l4*l5*l6)
LT.append(l1*l2*l3*l4*l5 + l1*l2*l3*l4*l6 + l1*l2*l3*l5*l6 + l1*l2*l4*l5*l6 + l1*l3*l4*l5*l6 + l2*l3*l4*l5*l6)
LT.append(l1*l2*l3*l5 + l1*l2*l4*l5 + l1*l3*l4*l5 + l2*l3*l4*l5 + l1*l2*l3*l6 + l1*l2*l4*l6 + l1*l3*l4*l6 + l2*l3*l4*l6 + l1*l3*l5*l6 + l2*l3*l5*l6 + l1*l4*l5*l6 + l2*l4*l5*l6)
LT.append(l1*l3*l5 + l2*l3*l5 + l1*l4*l5 + l2*l4*l5 + l1*l3*l6 + l2*l3*l6 + l1*l4*l6 + l2*l4*l6)
LT.append(l1*l2 + l3*l4 + l5*l6)
LT.append(l1 + l2 + l3 + l4 + l5 + l6)

In [ ]:
M.invariant_generators()

In [ ]:
S=R.quotient([R(t) for t in M.invariant_generators()])
for g in LT:
    S(g).lift()

In [ ]:
S=R.quotient(LT)
for g in M.invariant_generators():
    S(g).lift()

In [ ]:
#def wreath(d,N,D):
N=2
D=4
SND = SymmetricGroup(N*D)
SN = SymmetricGroup(N)
SD = SymmetricGroup(D)
#R.<l1,l2,l3,l4,l5,l6> = PolynomialRing(QQ,N*D)
R = PolynomialRing(QQ,N*D, 'l')
R.inject_variables()
f = l1*l3*l5 + l2*l3*l5 + l1*l4*l5 + l2*l4*l5 + l1*l3*l6 + l2*l3*l6 + l1*l4*l6 + l2*l4*l6
f=-(- l1*l2*l3*l5 - l1*l2*l4*l5 - l1*l3*l4*l5 - l2*l3*l4*l5 - l1*l2*l3*l6 - l2*l3*l4*l6 - l1*l3*l5*l6 - l2*l3*l5*l6 - l1*l4*l5*l6 -l2*l4*l5*l6)
G=[]
vars = [[R.gen(i+N*j) for i in range(N)] for j in range(D)]
d = [1..D]
n = [1..N]
num = 0
for sd in SD:
    new_vars = [vars[i-1] for i in sd(d)]
    for sn in SN:
        new_vars = [[t[i-1] for i in sn(n)] for t in new_vars]
        #print h,g, new_vars
        phi = R.hom(new_vars,R)
        #print phi
        L=[R.gens().index(t)+1 for t in phi.im_gens()]
        G.append(SND(L))

In [ ]:
M=MatrixGroup([g.matrix() for g in G])

In [ ]:
M.cardinality()

In [ ]:
IN=M.invariant_generators()

In [ ]:
R.<x1,x2,y1,y2,t,s1,s2,u1,u2>=QQ[]
a1 = t^2 - t*(x1+y1) + (x1)*(y1)
a2 = t^2 - t*(x1+y2) + (x1)*(y2)
a3 = t^2 - t*(x2+y1) + (x2)*(y1)
a4 = t^2 - t*(x2+y2) + (x2)*(y2)
a5 = t^2 #(0,1,0)
a6 = t^2 #(1,0,0)
a7 = t^2-2*t #(1,1,0)

In [ ]:
p=a1*a2*a3*a4*a5*a6*a7
Q = R.quotient((s1-(x1+x2), s2-(x1*x2), u1-(y1+y2), u2-(y1*y2))) ; Q
p2=Q(p).lift()
for n in range(14,-1,-1):
    p2.coefficient(t**n)

In [ ]:
R.<x1,x2,x3,y1,y2,y3,t,s1,s2,s3,u1,u2,u3>=QQ[]
a1 = t^2 - t*(x1+y1) + (x1)*(y1)
a2 = t^2 - t*(x1+y2) + (x1)*(y2)
a3 = t^2 - t*(x1+y3) + (x1)*(y3)
a4 = t^2 - t*(x2+y1) + (x2)*(y1)
a5 = t^2 - t*(x2+y2) + (x2)*(y2)
a6 = t^2 - t*(x2+y3) + (x2)*(y3)
a7 = t^2 - t*(x3+y1) + (x3)*(y1)
a8 = t^2 - t*(x3+y2) + (x3)*(y2)
a9 = t^2 - t*(x3+y3) + (x3)*(y3)
a10 = t^2 #(0,1,0)
a11 = t^2 #(1,0,0)
a12 = t^2-3*t #(1,1,0)
a13 = t^2-3*t #(-1,1,0)

In [ ]:
def check(self,n):
    f = self.change_ring(QQbar)
    per = f.periodic_points(n)
    mul = []
    t=1
    for Q in per:
        t=t*f.multiplier(Q,n).characteristic_polynomial()
        c=f.multiplier(Q,n).characteristic_polynomial().coefficients(sparse=False)
        c.pop(-1) #always monic
        mul.append(c)
    sig=[]
    e = SymmetricFunctions(QQbar).e()

    N = len(mul)
    R = f.base_ring()
    for i in range(0,N):
        T=[]
        for k in range(len(mul[0])):
            T.append(QQ(R(e([i+1]).expand(N)([m[k] for m in mul]))))
        sig.append(T)
    return sig,t

In [ ]:
%%time
P.<x,y,z,w> = ProjectiveSpace(QQ,3)
F = DynamicalSystem([4*x^2-3*z^2, 4*y^2-3*x*w,4*z^2+x*y, w^2])
S=sigma(F,1, deform=0)

In [ ]:
%%time
P.<x,y,z> = ProjectiveSpace(QQ,2)
F = DynamicalSystem([4*x^3-3*y*z^2, 4*y^3-3*x^3+z^3,4*z^3])
S=sigma(F,1, deform=1)

In [ ]:
S.<c,d>=QQ[]
t.<t>=S[]
a1 = t^2 +4*t-10
a2 = t^2 + (32*c*d + 16*(c+d))*t + (8*(c+d)+36)
a3 = t^2 - 64*c*d*t - 48*(c+d) - 56
a4 = t^2 + 256*c^2*d^2*t + 16*(c^2+d^2) - 32*c*d + 96*(c+d) + 32
a5 = t^2 -32*(c^2+d^2) + 64*c*d - 64*(c+d)
a6 = t^2
a7 = t^2
a = [a1,a2,a3,a4,a5,a6,a7]
prod(a)

In [ ]:
read "C:\\Users\\hutzba\\Dropbox\\VBShare\\multires.tex"
A := spresultant([x^2*y+x*y^2, x^2*z+x*z^2, y^2*z+y*z^2, -8*x*y*z+4*t*x*y+4*t*x*z+4*t*y*z+(-2*t^2)*x+(-2*t^2)*y+(-2*t^2)*z+t^3], [x, y, z]);
det(A[1]);

In [ ]:
set_verbose(None)
P.<x,y,z>=ProjectiveSpace(QQbar,2)
A=-8*x*y*z
B= 4*(x*y+y*z+x*z)
C=-2*(x+y+z)
f=DynamicalSystem([x^2+y^2,y^2,z^2+x^2])
X = f.periodic_points(1)

In [ ]:
r.<x0,x1,x2,x3,c,d>=QQ[]
I=r.ideal([x0-4,x1-(32*c*d + 16*(c + d)), x2-(-64*c*d),x3-(256*c^2*d^2)])
I.elimination_ideal((c,d))

In [ ]:
r.<x0,x1,x2,x3,x4,c,d>=QQ[]
I=r.ideal([x0-(-10),x1-(8*(c + d) + 36), x2-(-48*(c+d) - 56),x3-(16*(c^2+d^2) - 32*c*d + 96*(c + d) + 32), x4-(-32*(c^2+d^2) + 64*c*d - 64*(c+ d))])
I.elimination_ideal((c,d))

In [ ]:
set_verbose(None)
R.<c,d>=QQ[]
P.<x,y,z>=ProjectiveSpace(FractionField(R),2)
H=End(P)
f=H([x^2+c*z^2,y^2+d*z^2,z^2])
s=sigma(f,1)

In [ ]:
s

In [ ]:
w,t = s.parent().gens()
s1=s.parent()(s/(w-t^2))
D=(s1.discriminant(w))

In [ ]:
I = R.ideal(D.coefficients())
I.imension()

In [ ]:
R.<c,d>=QQ[]
P.<x,y,z>=ProjectiveSpace(FractionField(R),2)
H=End(P)
f=H([x^2+c^2*z^2,y^2+d^2*z^2,z^2])
m=matrix(FractionField(R),3,3,[0,d,0,c,0,0,0,0,c/d])
f.conjugate(m)

In [ ]:
P=f.domain()
f([1,0,0])

In [ ]:
f.multiplier(P([1,1,0]),1)